# Exploração Analítica dos Dados - Respostas às Perguntas de Negócio

Esta etapa tem como objetivo investigar, de forma estruturada e orientada ao negócio, os dados da camada Gold para responder diretamente às **três perguntas essenciais** definidas para o MVP.  
Aqui desenvolvemos análises profundas, baseadas em estatísticas, consultas SQL/Python e visualizações, para revelar padrões de comportamento nos anúncios do Airbnb em **Rio de Janeiro** e **New York City**.

---

## Pergunta 1 - O que diferencia anúncios de alta performance?

Objetivo:  
Identificar padrões que caracterizam os anúncios com melhor desempenho no Airbnb, considerando tanto características físicas do imóvel quanto comportamento de demanda, host e localização.

Serão investigados:

- distribuição de `estimated_occupancy_l365d` e `estimated_revenue_l365d`;
- perfil dos anúncios acima dos percentis 75 e 90;
- relação entre performance e:
  - tipo de imóvel (`room_type`, `property_type`);
  - capacidade (`accommodates`, `bedrooms`, `beds`);
  - comportamento do host (`host_is_superhost_bool`, tamanho do portfólio);
  - reviews e engajamento (`review_scores_rating`, `reviews_per_month`, `review_recency_days`);
  - localização (bairros e possíveis clusters de alta performance).

Essa pergunta integra elementos que, originalmente, estavam distribuídos nas análises de performance, reviews e comportamento dos hosts.

---

## Pergunta 2 - Quais fatores influenciam o preço da diária e a receita estimada?

Objetivo:  
Entender quais características explicam a variação de preço entre anúncios e como essas diferenças se refletem em receita estimada.

Serão analisados:

- distribuição de `price_numeric` por cidade;
- variação de preço por tipo de imóvel e capacidade;
- relação entre preço e:
  - reviews (volume, nota e recência);
  - status de superhost (`host_is_superhost_bool`);
  - tamanho do portfólio do host (indicador de profissionalização);
- análise de faixas de preço (buckets) e presença de outliers;
- comparação entre os mercados do Rio de Janeiro e de New York City.

Esta pergunta incorpora elementos da antiga pergunta sobre preço e parte da análise de reviews.

---

## Pergunta 3 - Como a localização afeta preço, demanda e competitividade?

Objetivo:  
Avaliar o impacto geográfico na performance e na precificação dos anúncios.

Serão investigados:

- distribuição espacial dos anúncios por bairro;
- diferenças de preço por localização;
- diferenças de ocupação estimada entre bairros;
- identificação de clusters locais de alta performance;
- comparação entre padrões geográficos de Rio de Janeiro e New York City.

Esta pergunta consolida a dimensão espacial do MVP, conectando localização com preço e desempenho.



# Pergunta 1 - O que diferencia anúncios de alta performance?

### Objetivo da análise

Identificar padrões que caracterizam os anúncios com melhor desempenho no Airbnb, considerando:
- ocupação estimada nos últimos 365 dias (`estimated_occupancy_l365d`);
- precificação (`price_numeric`);
- tipo de imóvel (`room_type`, `property_type`);
- capacidade (`accommodates`);
- comportamento do host (`host_type`, `host_is_superhost_bool`);
- reviews e engajamento;
- localização (bairro).

A meta é entender **quais atributos aparecem com maior frequência nos anúncios de alta performance**, permitindo comparar Rio de Janeiro e NYC com precisão.

---

### Estratégia da análise

1. Definir a métrica base de performance:
   - ocupação (`estimated_occupancy_l365d`)

2. Identificar:
   - anúncios acima do percentil 75 (alto desempenho);
   - anúncios acima do percentil 90 (top performers).

3. Comparar os anúncios de alta performance com o restante do mercado em termos de:
   - tipo de imóvel,
   - capacidade,
   - preço,
   - reviews,
   - localização,
   - perfil do host.

4. Extrair padrões claros que respondam à pergunta de negócio.




In [0]:
import pandas as pd
import numpy as np

# 1) Puxar os dados relevantes da dim_listings via SQL
query_occupancy = """
SELECT
  city,
  estimated_occupancy_l365d
FROM mvp_airbnb_catalog.default.dim_listings
WHERE estimated_occupancy_l365d IS NOT NULL
"""

df_occupancy_spark = spark.sql(query_occupancy)

# 2) Converter para pandas para análise em Python
df_occupancy = df_occupancy_spark.toPandas()

# 3) Função auxiliar para calcular percentis em grupo
def percentile(series, q):
    return np.percentile(series, q)

# 4) Calcular estatísticas por cidade
summary_by_city = df_occupancy.groupby("city")["estimated_occupancy_l365d"].agg(
    count="count",
    min_value="min",
    median=lambda s: np.percentile(s, 50),
    p75=lambda s: np.percentile(s, 75),
    p90=lambda s: np.percentile(s, 90),
    max_value="max",
    mean_value="mean"
).reset_index()

summary_by_city


In [0]:
# Extrair apenas os thresholds p75 e p90 como dicionário de referência
thresholds = {}

for _, row in summary_by_city.iterrows():
    city = row["city"]
    thresholds[city] = {
        "p75": row["p75"],
        "p90": row["p90"]
    }

thresholds

### Estatísticas de Ocupação - Interpretação dos Resultados

Os valores calculados em Python para `estimated_occupancy_l365d` revelam diferenças importantes entre Rio de Janeiro e New York City:

| Cidade           | Total Listings | Mediana | p75  | p90  | Máximo | Média |
|------------------|----------------|---------|------|------|--------|-------|
| New York City    | 36.111         | 0       | 60   | 240  | 255    | 46.95 |
| Rio de Janeiro   | 43.068         | 18      | 84   | 174  | 255    | 54.50 |

#### Principais observações:

1. **Distribuição muito assimétrica em ambas as cidades**  
   A mediana extremamente baixa (0 em NYC e 18 no Rio) indica que:
   - a maioria dos anúncios tem baixa ocupação estimada,
   - existe uma “cauda longa” de poucos anúncios que ocupam muito ao longo do ano.

2. **Os percentis 75 e 90 são razoavelmente altos**, mostrando que:
   - só 25% dos anúncios atingem ocupações acima de 60 dias/ano em NYC e 84 dias/ano no Rio,
   - o Top 10% atinge valores muito superiores (p90 = 240 em NYC e 174 no Rio).

3. **NYC parece ter um comportamento mais polarizado**  
   - mediana em zero → muitos imóveis sem atividade detectable,
   - p90 muito alto (240), chegando perto do máximo absoluto (255),
   - média menor que a do Rio, sugerindo concentração de alta performance em poucos imóveis.

4. **O Rio tem uma distribuição menos concentrada**  
   - mediana maior,
   - p75 maior,
   - média superior,
   - indicando uma base mais distribuída, com mais anúncios gerando algum nível de ocupação.

---

### Definição de “Alta Performance” por Cidade

Com base nos resultados, vamos usar os percentis como thresholds:

#### Rio de Janeiro
- **Alta performance** → ocupação > **84 dias/ano** (p75)  
- **Top performers** → ocupação > **174 dias/ano** (p90)

#### New York City
- **Alta performance** → ocupação > **60 dias/ano** (p75)  
- **Top performers** → ocupação > **240 dias/ano** (p90)

Esses limites respeitam:
- o comportamento empírico de cada mercado,
- a cauda longa presente na distribuição,
- e a metodologia baseada em estatística robusta (percentis).

---

### Próximo passo

Agora que definimos formalmente os thresholds, vamos construir duas novas tabelas analíticas:

1. **Anúncios acima e abaixo do p75**, por cidade  
2. **Top 10% (p90)**, por cidade




### Segmentação dos anúncios por nível de performance

A partir dos percentis de `estimated_occupancy_l365d` calculados por cidade, vamos classificar cada anúncio em três grupos:

- **Base** → anúncios com ocupação até o percentil 75 da sua cidade  
- **Alta performance** → anúncios com ocupação acima do p75 da sua cidade  
- **Top performers** → anúncios com ocupação acima do p90 da sua cidade  

Essa segmentação será utilizada para comparar o perfil dos anúncios de alta performance com o restante do mercado em termos de:

- tipo de imóvel,
- capacidade,
- preço,
- reviews,
- perfil do host,
- localização.


In [0]:
# Puxar as colunas relevantes da dim_listings
query_listings_perf = """
SELECT *
FROM mvp_airbnb_catalog.default.dim_listings
WHERE estimated_occupancy_l365d IS NOT NULL
"""

df_listings_spark = spark.sql(query_listings_perf)
df_listings = df_listings_spark.toPandas()


# Criar coluna performance_group
def classify_performance(row):
    city = row["city"]
    occ = row["estimated_occupancy_l365d"]
    p75 = thresholds[city]["p75"]
    p90 = thresholds[city]["p90"]

    if occ > p90:
        return "top_performer"
    elif occ > p75:
        return "high_perf"
    else:
        return "base"

df_listings["performance_group"] = df_listings.apply(classify_performance, axis=1)

# flags auxiliares
df_listings["is_high_perf"] = df_listings["performance_group"].isin(["high_perf", "top_performer"])
df_listings["is_top_performer"] = df_listings["performance_group"] == "top_performer"


# Distribuição dos grupos por cidade
perf_summary = (
    df_listings
    .groupby(["city", "performance_group"])
    .size()
    .reset_index(name="count")
)

perf_summary


### Análise - Distribuição dos anúncios por nível de performance

#### Objetivo

Segmentar os anúncios em grupos de performance, com base na métrica `estimated_occupancy_l365d`, para entender como o mercado se distribui entre anúncios de baixa, média e alta performance em cada cidade.

A classificação foi feita a partir de **percentis por cidade**:

- **Base Performance** → anúncios com ocupação **até o percentil 75** da sua cidade  
- **High Performance** → anúncios com ocupação **entre o percentil 75 e o percentil 90**  
- **Top Performers** → anúncios com ocupação **acima do percentil 90**

Essa segmentação é relativa a cada cidade, respeitando a distribuição de ocupação de cada mercado.

---

#### Resultados - Contagem por grupo e cidade

| Cidade           | Grupo            | Quantidade de anúncios |
|------------------|------------------|------------------------|
| New York City    | Base Performance | 28.470                 |
| New York City    | High Performance | 4.401                  |
| New York City    | Top Performers   | 3.240                  |
| Rio de Janeiro   | Base Performance | 32.691                 |
| Rio de Janeiro   | High Performance | 6.076                  |
| Rio de Janeiro   | Top Performers   | 4.301                  |

---

#### Interpretação

- A maior parte dos anúncios, em ambas as cidades, está no grupo **Base Performance**, o que é esperado quando usamos thresholds baseados em percentis (apenas uma parcela menor da distribuição entra nos grupos superiores).
- Tanto em **NYC** quanto no **Rio de Janeiro**, existe um grupo relativamente pequeno de **Top Performers** (acima do p90), que representa os anúncios mais bem-sucedidos em termos de ocupação ao longo dos últimos 365 dias.
- O fato de as duas cidades apresentarem três faixas bem definidas permite, a partir de agora, **comparar o perfil desses grupos** (tipo de imóvel, capacidade, preço, reviews, superhost etc.) para responder diretamente à pergunta:

> **O que diferencia os anúncios de alta performance em relação aos demais?**

Nas próximas etapas, utilizaremos essa segmentação (`performance_group`) para comparar as características dos anúncios entre os grupos.


### Comparação entre anúncios por nível de performance  
#### (Base Performance vs High Performance vs Top Performers)

Agora que classificamos todos os anúncios em três grupos de performance - com base na métrica `estimated_occupancy_l365d` - o próximo passo é investigar **quais características diferenciam os anúncios de alta performance do restante do mercado**.

Essa análise responde diretamente à primeira pergunta de negócio do MVP:

> **O que diferencia um anúncio de alta performance?**

Para isso, examinaremos um conjunto de atributos-chave:

1. **Tipo de imóvel (`property_type` e `room_type`)**  
   Avaliaremos se certas categorias estruturais aparecem com mais frequência entre os Top Performers.

2. **Capacidade (`accommodates`, `bedrooms`, `beds`)**  
   Vamos verificar se anúncios maiores, menores ou de determinadas configurações são mais bem-sucedidos.

3. **Preço (`price_numeric`)**  
   Comparamos o posicionamento de preço entre os grupos de performance em cada cidade.

4. **Reviews (`review_scores_rating`, `reviews_per_month`)**  
   Investigaremos se os anúncios de alta performance também possuem:
   - notas mais altas,  
   - maior fluxo de reviews,  
   - perfil de engajamento mais forte.

5. **Perfil do host (`host_is_superhost_bool`, `host_identity_verified_bool`)**  
   Avaliaremos se características do host estão correlacionadas com melhor ocupação.

6. **Localização (`neighbourhood_cleansed`)**  
   Verificaremos se certos bairros têm maior concentração de anúncios de alta performance.



### Análise - Diferenças de Performance por Tipo de Imóvel  
#### (Room Type e Property Type)

#### Objetivo da análise
Investigar se determinados tipos de imóveis apresentam maior probabilidade de alcançar alta performance no Airbnb.  
A análise compara os três grupos de performance (`Base`, `High Performance`, `Top Performers`) nas duas cidades - Rio de Janeiro e New York City.

Os atributos analisados são:

- **room_type** (categoria do espaço oferecido)  
- **property_type** (tipo estrutural/imobiliário do anúncio)

Essa etapa busca responder:

> Quais tipos de imóveis são mais comuns entre os anúncios de alta performance?


In [0]:
# Garantir que o df_listings é uma cópia de trabalho
df = df_listings.copy()

# Vamos calcular percentuais por tipo de imóvel dentro de cada cidade × performance_group
group_cols = ['city', 'performance_group', 'room_type']
df_room = (
    df.groupby(group_cols)
      .size()
      .reset_index(name='count')
)

# Percentual dentro de cada cidade × grupo de performance
df_room['pct'] = df_room.groupby(['city', 'performance_group'])['count'] \
                        .transform(lambda x: x / x.sum() * 100)

# Pivot para visualização
pivot_room = df_room.pivot_table(
    index=['city', 'room_type'],
    columns='performance_group',
    values='pct',
    aggfunc='sum'
).reset_index()

pivot_room


### Análise - Diferenças de Performance por Tipo de Imóvel  
#### `room_type`

#### Objetivo
Avaliar se determinados tipos de acomodação apresentam maior presença entre os anúncios de alta performance.  
A análise compara a distribuição percentual dos room_types em cada grupo de performance (Base, High Performance, Top Performers) para Rio de Janeiro e New York City.

---

### Resultados - New York City

| Room Type         | Base (%) | High Perf (%) | Top Performer (%) |
|------------------|-----------|----------------|--------------------|
| Entire home/apt  | 52.83     | 60.35         | 45.40              |
| Private room     | 45.32     | 38.95         | 53.49              |
| Shared room      | 0.66      | 0.50          | 0.99               |
| Hotel room       | 1.19      | 0.20          | 0.12               |

**Pontos observados:**
- Entire home/apt apresenta maior representação no grupo High Performance.
- Private rooms assumem maior proporção no grupo Top Performer, indicando que algumas unidades desse tipo atingem desempenho elevado no contexto de NYC.
- Categorias Shared e Hotel room têm participação marginal em todos os níveis de performance.

---

### Resultados - Rio de Janeiro

| Room Type         | Base (%) | High Perf (%) | Top Performer (%) |
|------------------|-----------|----------------|--------------------|
| Entire home/apt  | 76.69     | 90.77         | 90.86              |
| Private room     | 22.03     | 8.87          | 8.88               |
| Shared room      | 1.21      | 0.31          | 0.25               |
| Hotel room       | 0.06      | 0.05          | 0.00               |

**Pontos observados:**
- Entire home/apt domina amplamente os grupos High Performance e Top Performer.
- Private rooms têm participação reduzida nos grupos superiores.
- Shared e Hotel rooms aparecem com frequência muito baixa nesses segmentos.

---

### Conclusão da Análise
- Em ambas as cidades, imóveis classificados como Entire home/apt apresentam maior concentração nos grupos de performance superior.
- No Rio de Janeiro, essa predominância é mais intensa, com forte concentração dessa categoria entre os anúncios de melhor desempenho.
- Em New York City, há maior heterogeneidade: Entire homes são mais frequentes no grupo de alta performance, enquanto private rooms também aparecem com proporção relevante entre os Top Performers.
- Shared e Hotel rooms possuem baixa representatividade nas faixas de maior performance.


### Análise - Diferenças de Performance por Capacidade  
#### (accommodates, bedrooms, beds)

#### Objetivo
Avaliar se a capacidade dos imóveis - representada pelas variáveis `accommodates`, `bedrooms` e `beds` - está associada a maior presença nos grupos de alta performance.  
A análise compara a distribuição dessas características entre Base Performance, High Performance e Top Performers, separando por cidade.

Os atributos analisados são:

- **accommodates** - quantas pessoas o imóvel comporta  
- **bedrooms** - quantidade de quartos  
- **beds** - quantidade de camas  

Com essa análise, buscamos responder:

> Imóveis com maior capacidade tendem a performar melhor no Airbnb?


In [0]:
# Copiando o df que iremos analisar
df_cap = df_listings.copy()

# Selecionar apenas colunas que vamos usar
cols = [
    "city",
    "performance_group",
    "accommodates",
    "bedrooms",
    "beds"
]

df_cap = df_cap[cols].dropna(subset=["accommodates"])

# Estatísticas por cidade e performance_group
summary_capacity = (
    df_cap
    .groupby(["city", "performance_group"])
    .agg(
        count_listings=("accommodates", "count"),
        accommodates_mean=("accommodates", "mean"),
        accommodates_median=("accommodates", "median"),
        accommodates_min=("accommodates", "min"),
        accommodates_max=("accommodates", "max"),
        bedrooms_mean=("bedrooms", "mean"),
        bedrooms_median=("bedrooms", "median"),
        beds_mean=("beds", "mean"),
        beds_median=("beds", "median"),
    )
    .reset_index()
)

summary_capacity


In [0]:
# Definir faixas de capacidade
bins = [0, 2, 4, 6, 8, 10, 100]
labels = ["1-2", "3-4", "5-6", "7-8", "9-10", "11+"]

df_cap["accommodates_bucket"] = pd.cut(
    df_cap["accommodates"],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

# Distribuição percentual por faixa de capacidade, cidade e grupo de performance
bucket_dist = (
    df_cap
    .groupby(["city", "performance_group", "accommodates_bucket"])
    .size()
    .reset_index(name="count")
)

bucket_dist["pct"] = bucket_dist.groupby(["city", "performance_group"])["count"] \
                                .transform(lambda x: x / x.sum() * 100)

bucket_dist.sort_values(["city", "performance_group", "accommodates_bucket"], inplace=True)

bucket_dist

### Resultados - Estatísticas Descritivas

#### New York City

| Grupo            | Accommodates (Média) | Mediana | Quartos (Média) | Camas (Média) |
|------------------|-----------------------|---------|------------------|----------------|
| Base             | 2.71                  | 2       | 1.43             | 1.59           |
| High Performance | 2.75                  | 2       | 1.23             | 1.63           |
| Top Performers   | 2.88                  | 2       | 1.25             | 1.76           |

**Pontos observados:**
- Os três grupos apresentam capacidade semelhante, com média próxima de 2–3 hóspedes.
- Anúncios de alta performance apresentam **ligeiro aumento** no número médio de camas.
- Bedrooms permanecem majoritariamente em 1 quarto em todos os níveis de performance.
- A capacidade máxima chega a 16 hóspedes em todos os grupos, mas representa casos raros.

---

#### Rio de Janeiro

| Grupo            | Accommodates (Média) | Mediana | Quartos (Média) | Camas (Média) |
|------------------|-----------------------|---------|------------------|----------------|
| Base             | 3.98                  | 4       | 1.56             | 2.40           |
| High Performance | 3.93                  | 4       | 1.43             | 2.34           |
| Top Performers   | 3.81                  | 4       | 1.32             | 2.22           |

**Pontos observados:**
- Os três grupos possuem mediana uniforme de 4 hóspedes.
- A média de capacidade diminui levemente conforme a performance aumenta.
- O número de quartos e camas também apresenta queda gradual nos grupos superiores.
- Isso sugere que, no Rio, imóveis muito grandes não são predominantes entre os anúncios de melhor desempenho. Ponto semelhante quando analisamos NYC.

---

### Distribuição por Faixas de Capacidade (accommodates)

#### New York City

| Faixa | Base (%) | High Perf (%) | Top Performer (%) |
|-------|-----------|----------------|--------------------|
| 1–2   | 65.83     | 63.35         | 68.24             |
| 3–4   | 23.27     | 25.27         | 16.76             |
| 5–6   | 7.35      | 8.16          | 9.66              |
| 7–8   | 2.23      | 2.04          | 3.64              |
| 9–10  | 0.58      | 0.68          | 1.02              |
| 11+   | 0.74      | 0.50          | 0.68              |

**Pontos observados:**
- O grupo Top Performer concentra mais propriedades **1–2 hóspedes**.
- Propriedades maiores (5+ hóspedes) aparecem nos grupos superiores, mas em proporções pequenas.
- A distribuição reforça que o mercado de NYC é majoritariamente composto por unidades compactas, mesmo no topo da performance.

---

#### Rio de Janeiro

| Faixa | Base (%) | High Perf (%) | Top Performer (%) |
|-------|-----------|----------------|--------------------|
| 1–2   | 32.46     | 27.81         | 28.97             |
| 3–4   | 40.84     | 46.86         | 47.45             |
| 5–6   | 17.78     | 18.45         | 17.79             |
| 7–8   | 4.71      | 4.36          | 3.86              |
| 9–10  | 1.95      | 1.33          | 1.14              |
| 11+   | 2.26      | 1.18          | 0.79              |

**Pontos observados:**
- No Rio, a faixa **3–4 hóspedes** domina os grupos de alta performance.
- A participação de imóveis muito grandes (7+ hóspedes) diminui nos grupos superiores.
- Isso sugere que unidades de porte médio são mais competitivas no contexto carioca.

---

### Conclusão da Análise
- **New York City:**  
  Os anúncios de alta performance tendem a ser unidades pequenas (até 2 hóspedes), indicando forte competitividade entre imóveis compactos.  
  A capacidade maior aparece nos grupos superiores, mas com participação reduzida.

- **Rio de Janeiro:**  
  O desempenho superior está concentrado em imóveis de porte médio (3–4 hóspedes), enquanto unidades muito grandes têm presença menor nas faixas de alta performance.

- **Visão geral da etapa:**  
  Não há evidências, em nenhuma das cidades, de que imóveis maiores superem imóveis compactos ou médios em termos de performance.  
  A relação entre capacidade e performance varia de acordo com a dinâmica de cada cidade, refletindo preferências específicas do mercado local.



### Análise - Diferenças de Performance por Preço  
#### (price_numeric)

#### Objetivo
Avaliar se a variável `price_numeric` está associada aos diferentes níveis de performance dos anúncios, comparando como o preço se distribui entre Base Performance, High Performance e Top Performers.

A análise será feita separadamente para:

- **New York City** - preços em USD  
- **Rio de Janeiro** - preços em BRL  

Os resultados indicarão se imóveis mais caros tendem a ter melhor ocupação ou se há faixa de preço mais competitiva dentro de cada mercado.

Serão examinados:

- estatísticas descritivas por grupo (média, mediana, min, max, percentis);
- distribuição em faixas de preço (buckets);
- padrões distintos entre as duas cidades.

Essa etapa responde à pergunta:

> Os anúncios de alta performance são mais caros, mais baratos ou têm distribuição de preço semelhante aos demais?



### Análise - Diferenças de Performance por Preço  
#### `price_numeric`

#### Objetivo
Avaliar a relação entre preço (`price_numeric`) e desempenho dos anúncios, comparando os grupos Base Performance, High Performance e Top Performers em cada cidade.  
A análise busca identificar se anúncios de alta performance tendem a praticar preços mais altos, mais baixos ou distribuídos de forma distinta.

#### Nota sobre o tratamento dos dados
Durante a etapa de Qualidade dos Dados, foi identificado que parte dos anúncios possui `price_numeric` nulo.  
Esses valores representam ausência de informação, e não preços reais, e portanto **não devem ser utilizados para análises de distribuição ou estatísticas de preço**.

Seguindo a decisão documentada naquela etapa, os registros com `price_numeric` nulo serão removidos para esta análise, garantindo consistência e precisão nos resultados.

A análise incluirá:

- estatísticas descritivas (média, mediana, percentis);  
- distribuição de preços por faixas (buckets);  
- comparações separadas entre Rio de Janeiro e New York City.



In [0]:
# Filtrar apenas registros com preço válido (conforme etapa de Qualidade)
df_price_valid = df_listings.copy()

# Filtrar apenas registros com preço válido (conforme etapa de Qualidade)
df_price_valid = df_price_valid[df_price_valid["has_valid_price"] == True]

# Garantir que preço é numérico
df_price_valid = df_price_valid.dropna(subset=["price_numeric"])

# Estatísticas descritivas por cidade e performance_group
summary_price = (
    df_price_valid
    .groupby(["city", "performance_group"])
    .agg(
        count_listings=("price_numeric", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_min=("price_numeric", "min"),
        price_max=("price_numeric", "max"),
        p25=("price_numeric", lambda x: x.quantile(0.25)),
        p75=("price_numeric", lambda x: x.quantile(0.75)),
        p90=("price_numeric", lambda x: x.quantile(0.90)),
    )
    .reset_index()
)

summary_price


In [0]:
# Definição de faixas de preço
bins = [0, 100, 200, 300, 500, 1000, 10000]
labels = ["0-100", "101-200", "201-300", "301-500", "501-1000", "1001+"]

df_price_valid["price_bucket"] = pd.cut(
    df_price_valid["price_numeric"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

bucket_dist_price = (
    df_price_valid
    .groupby(["city", "performance_group", "price_bucket"])
    .size()
    .reset_index(name="count")
)

bucket_dist_price["pct"] = bucket_dist_price.groupby(["city", "performance_group"])["count"] \
                                            .transform(lambda x: x / x.sum() * 100)

bucket_dist_price.sort_values(["city", "performance_group", "price_bucket"], inplace=True)

bucket_dist_price


### Análise - Diferenças de Performance por Preço  
#### `price_numeric`

#### Resultados - Estatísticas Descritivas

Os resultados consolidados para cada cidade e grupo de performance indicam diferenças claras na distribuição de preços.

**New York City**

| Performance Group | Count | Mediana | Média | P25 | P75 | P90 |
|-------------------|-------|---------|-------|-----|-----|------|
| Base              | 14.401 | 155 | 896.85 | 81  | 301 | 528 |
| High Performance  | 3.890  | 141 | 207.99 | 86  | 216 | 351 |
| Top Performer     | 3.037  | 170 | 260.00 | 115 | 278 | 425 |

**Rio de Janeiro**

| Performance Group | Count | Mediana | Média | P25 | P75 | P90 |
|-------------------|-------|---------|-------|-----|-----|------|
| Base              | 28.699 | 346 | 834.35 | 212 | 638 | 1321 |
| High Performance  | 5.815  | 270 | 395.64 | 190 | 423 | 689.6 |
| Top Performer     | 4.156  | 258 | 361.28 | 183 | 393 | 635.5 |

#### Interpretação

- Em ambas as cidades, os anúncios de **alta performance** apresentam uma **faixa de preço mais concentrada** e menor dispersão que os anúncios base.  
- Em NYC, anúncios Top Performer são levemente mais caros na mediana, mas com amplitude menor.  
- No Rio, os grupos High Performance e Top Performer apresentam preços mais moderados, com medianas menores que a do grupo Base.

A combinação destes elementos indica diferenças estruturais no posicionamento de preço para cada nível de performance em cada mercado.

---

### Distribuição por Faixas de Preço (Buckets)

Os buckets evidenciam como a precificação se distribui entre os anúncios.

**New York City**

- No grupo Base, o mercado é distribuído principalmente entre:
  - 0–100 USD (33%)
  - 101–200 USD (28%)
  - 201–300 USD e 301–500 USD (aprox. 14% cada)
- High Performance concentra-se em:
  - 101–200 USD (40%)
  - 0–100 USD (32%)
- Top Performers deslocam-se mais fortemente para:
  - 101–200 USD (41,6%)
  - 201–300 USD (19%)
  - 301–500 USD (14,6%)

**Rio de Janeiro**

- O grupo Base possui maior diversidade e dispersão:
  - R$ 101–200 (19%)
  - R$ 201–300 (21%)
  - R$ 301–500 (23%)
  - R$ 501–1000 (19%)

- High Performance apresenta distribuição mais concentrada:
  - R$ 101–200 (26,7%)
  - R$ 201–300 (26,9%)
  - R$ 301–500 (24,6%)

- Top Performers seguem padrão semelhante:
  - R$ 101–200 (28,5%)
  - R$ 201–300 (28,7%)
  - R$ 301–500 (23,5%)

#### Interpretação

- Em NYC, anúncios de maior performance tendem a se posicionar de forma consistente na faixa **101–300 USD**, com pouca presença nas faixas extremas.
- No Rio, os melhores desempenhos aparecem em **faixas intermediárias (101–500 BRL)**, enquanto o grupo Base apresenta forte dispersão entre preços muito baixos e muito altos.

Essas diferenças sugerem estruturas de demanda distintas entre os mercados, especialmente no que diz respeito às expectativas de preço e faixa de competitividade.

---


### Análise - Impacto das Reviews na Performance dos Anúncios

#### Objetivo
Avaliar como diferentes métricas relacionadas a reviews se distribuem entre os grupos Base Performance, High Performance e Top Performers, identificando possíveis padrões que relacionam volume de reviews e qualidade da avaliação com o desempenho dos anúncios.

A análise examinará separadamente Rio de Janeiro e New York City, considerando:

- `review_scores_rating` → nota média dada pelos hóspedes  
- `reviews_per_month` → dinâmica de entrada de reviews  
- `review_recency_days` → tempo desde o último review (indicador de atividade)  
- indicadores de presença/ausência de reviews  

O objetivo é responder:

> Os anúncios de alta performance diferenciam-se pelo volume, atualidade e qualidade das avaliações?

Somente registros com dados válidos serão utilizados, de acordo com o tratamento estabelecido na etapa de Qualidade dos Dados.

In [0]:
# Copia do df_listings

df_rev = df_listings.copy()

# Estatisticas para a coluna review_score_rating
summary_rating = (
    df_rev
    .groupby(["city", "performance_group"])
    .agg(
        count_valid_rating=("review_scores_rating", lambda x: x.dropna().shape[0]),
        rating_mean=("review_scores_rating", "mean"),
        rating_median=("review_scores_rating", "median"),
        rating_p25=("review_scores_rating", lambda x: np.percentile(x.dropna(), 25) if x.dropna().shape[0] > 0 else np.nan),
        rating_p75=("review_scores_rating", lambda x: np.percentile(x.dropna(), 75) if x.dropna().shape[0] > 0 else np.nan)
    )
    .reset_index()
)

summary_rating

In [0]:
# Estatísticas para reviews_per_month
summary_rpm = (
    df_rev
    .groupby(["city", "performance_group"])
    .agg(
        count_valid_rpm=("reviews_per_month", lambda x: x.dropna().shape[0]),
        rpm_mean=("reviews_per_month", "mean"),
        rpm_median=("reviews_per_month", "median"),
        rpm_p75=("reviews_per_month", lambda x: np.percentile(x.dropna(), 75) if x.dropna().shape[0] > 0 else np.nan),
        rpm_p90=("reviews_per_month", lambda x: np.percentile(x.dropna(), 90) if x.dropna().shape[0] > 0 else np.nan)
    )
    .reset_index()
)

summary_rpm


In [0]:
# Estatísticas para review_recency_days
summary_recency = (
    df_listings
    .groupby(["city", "performance_group"])
    .agg(
        count_valid_recency=("review_recency_days", lambda x: x.dropna().shape[0]),
        recency_mean=("review_recency_days", "mean"),
        recency_median=("review_recency_days", "median"),
        recency_p25=("review_recency_days", lambda x: np.percentile(x.dropna(), 25) if x.dropna().shape[0] > 0 else np.nan),
        recency_p75=("review_recency_days", lambda x: np.percentile(x.dropna(), 75) if x.dropna().shape[0] > 0 else np.nan)
    )
    .reset_index()
)

summary_recency


### Análise - Impacto das Reviews na Performance dos Anúncios

#### Objetivo

Avaliar como métricas de avaliação dos hóspedes variam entre os grupos de performance (Base, High Performance e Top Performer) para identificar padrões de qualidade, demanda e atualidade dos anúncios.

As métricas analisadas foram:

- **review_scores_rating** → qualidade média das avaliações.
- **reviews_per_month** → velocidade com que novos reviews são gerados (indicador de demanda e visibilidade).
- **review_recency_days** → tempo desde o último review (indicador de atualidade e atividade recente).

---

### Resultados

#### 1. Qualidade das avaliações - `review_scores_rating`

| Cidade | Grupo | Média | Mediana | P25 | P75 |
|-------|--------|---------|---------|---------|---------|
| **NYC** | Base | 4.70 | 4.86 | 4.61 | 5.00 |
|        | High Perf | 4.77 | 4.86 | 4.67 | 5.00 |
|        | Top Performer | 4.80 | 4.85 | 4.72 | 4.93 |
| **RJ** | Base | 4.79 | 4.96 | 4.75 | 5.00 |
|        | High Perf | 4.85 | 4.89 | 4.79 | 4.95 |
|        | Top Performer | 4.87 | 4.89 | 4.81 | 4.95 |

**Interpretação:**  
Os grupos de maior performance apresentam avaliações ligeiramente superiores, mas as diferenças são pequenas. O rating não é o principal fator de diferenciação entre os níveis - o mercado tende a concentrar notas altas para a maioria dos anúncios.

---

#### 2. Dinâmica dos reviews - `reviews_per_month`

| Cidade | Grupo | Média | Mediana | P75 | P90 |
|--------|---------|---------|-----------|---------|---------|
| **NYC** | Base | 0.35 | 0.13 | 0.41 | 1.00 |
|         | High Perf | 0.89 | 0.49 | 1.21 | 2.01 |
|         | Top Performer | 3.22 | 2.62 | 3.78 | 5.40 |
| **RJ** | Base | 0.65 | 0.40 | 0.88 | 1.47 |
|         | High Perf | 1.91 | 1.79 | 2.30 | 2.91 |
|         | Top Performer | 3.22 | 3.02 | 3.92 | 5.00 |

**Interpretação:**  
A diferença mais clara entre os grupos aparece na **velocidade com que reviews novos chegam**.  
Top Performers recebem novos reviews **3 a 5 vezes mais rápido** que o restante do mercado.  
Isso indica:

- maior volume de reservas,
- maior visibilidade nos rankings,
- maior fidelidade ou fluxo de hóspedes.

---

#### 3. Atualidade do anúncio - `review_recency_days`

| Cidade | Grupo | Média (dias) | Mediana (dias) | P25 | P75 |
|--------|---------|----------------|----------------|---------|----------|
| **NYC** | Base | 1488 | 1031 | 552 | 2332 |
|         | High Perf | 151 | 126 | 98 | 189 |
|         | Top Performer | 93 | 83 | 74 | 97 |
| **RJ** | Base | 360 | 217 | 113 | 327 |
|         | High Perf | 103 | 91 | 81 | 110 |
|         | Top Performer | 91 | 84 | 78 | 92 |

**Interpretação:**  
A recência dos reviews é um dos diferenciais mais fortes entre os grupos.  
Top Performers têm avaliações **extremamente recentes**, enquanto anúncios de baixa performance podem ficar **meses ou até anos** sem receber um novo review.

Isso reforça que o fator “atividade atual” acompanha fortemente a performance operacional dos anúncios.

---

### Conclusão da Análise das Reviews

- **Qualidade (rating)** diferencia pouco - praticamente todos recebem notas altas.
- **Frequência de reviews (reviews_per_month)** é um dos indicadores mais fortes de performance.  
  Top Performers recebem novos reviews em ritmo muito superior.
- **Recência dos reviews** é outro sinal claro: anúncios de alta performance mantêm atividade constante, enquanto anúncios base ficam longos períodos sem avaliações.

Em conjunto, as métricas de review mostram que a performance está muito mais associada à **demanda ativa e fluxo de hóspedes** do que à simples qualidade média das avaliações.



### Análise - Comportamento e Perfil dos Hosts na Performance dos Anúncios

Nesta análise, investigamos como características dos anfitriões influenciam o desempenho dos anúncios no Airbnb.  
Nosso objetivo é identificar se existe um padrão de **profissionalização**, experiência ou consistência operacional que diferencie:

- anúncios de **baixa performance** (grupo “base” - até o percentil 75),
- anúncios de **alta performance** (percentil 75–90),
- anúncios **top performers** (percentil 90–100).

A análise considera tanto o mercado **do Rio de Janeiro** quanto o de **New York City**, permitindo observar diferenças estruturais e padrões comuns entre os dois ambientes.

### Aspectos avaliados nesta seção

- **Proporção de superhosts** por grupo de performance.  
  → Avalia se anfitriões com histórico comprovado de qualidade tendem a entregar anúncios com melhor performance.

- **Tamanho do portfólio do host** (`calculated_host_listings_count`).  
  → Mede se hosts com mais anúncios (indicando profissionalização) apresentam melhores métricas de performance.

- **Categorias de profissionalização** baseadas no tamanho do portfólio:  
  - *casual*: 1 anúncio  
  - *semi-profissional*: 2 a 5 anúncios  
  - *profissional*: 6 ou mais anúncios  

- **Distribuição dos grupos de performance** dentro dessas categorias.  
  → Identifica se top performers são mais frequentes entre hosts profissionais.

- **Comparação RJ × NYC**, destacando diferenças estruturais entre um mercado mais maduro (NYC) e outro mais heterogêneo (RJ).

O entendimento do comportamento dos hosts é fundamental para explicar **por que certos anúncios atingem níveis superiores de performance**, complementando análises anteriores de tipo de imóvel, capacidade, preço e reviews.  



In [0]:
# Cópia da base
df_hosts = df_listings.copy()

# Construindo o tamanho do portfólio (quantos anúncios cada host possui)
df_host_portfolio = (
    df_hosts
    .groupby(["city", "host_id"])
    .agg(
        host_listings_count=("listing_id", "nunique")
    )
    .reset_index()
)

# Juntando o portfólio na base de listings
df_hosts = df_hosts.merge(
    df_host_portfolio,
    on=["city", "host_id"],
    how="left"
)


# 1) Proporção de superhosts por grupo
summary_superhost = (
    df_hosts
    .groupby(["city", "performance_group"])
    .agg(
        total_listings=("listing_id", "count"),
        superhost_count=("host_is_superhost_bool", lambda x: x.sum())
    )
    .reset_index()
)

summary_superhost["pct_superhost"] = (
    summary_superhost["superhost_count"] / summary_superhost["total_listings"]
).round(3)


summary_superhost


In [0]:

# 2) Portfólio dos hosts por grupo de performance
summary_portfolio = (
    df_hosts
    .groupby(["city", "performance_group"])
    .agg(
        count_listings=("listing_id", "count"),
        portfolio_mean=("host_listings_count", "mean"),
        portfolio_median=("host_listings_count", "median"),
        portfolio_p75=("host_listings_count", lambda x: x.quantile(0.75)),
        portfolio_max=("host_listings_count", "max")
    )
    .round(2)
    .reset_index()
)
summary_portfolio

In [0]:

# 3) Classificação dos hosts (casual, semi, profissional)
def classify_host(n):
    if n == 1:
        return "casual"
    elif 2 <= n <= 5:
        return "semi_profissional"
    else:
        return "profissional"

df_hosts["host_category"] = df_hosts["host_listings_count"].apply(classify_host)

summary_categories = (
    df_hosts
    .groupby(["city", "host_category", "performance_group"])
    .agg(total_listings=("listing_id", "count"))
    .reset_index()
)

summary_categories


### Interpretação dos Resultados - Comportamento e Perfil dos Hosts

Os resultados mostram uma diferença clara no comportamento dos hosts entre os grupos de performance, tanto em **New York City** quanto no **Rio de Janeiro**. Os padrões gerais são semelhantes nas duas cidades, embora com intensidades diferentes.

---

### **1. Superhosts são desproporcionalmente mais comuns entre anúncios de alta performance**

A proporção de superhosts cresce de forma consistente conforme a performance aumenta:

| Cidade | Base | High Perf | Top Performer |
|-------|------|-----------|----------------|
| **NYC** | 10% | 52% | 63% |
| **RJ** | 23% | 64% | 73% |

**Principais conclusões:**

- Em NYC, a proporção sobe **de 10% para 63%**, indicando forte relação entre ser superhost e obter alta performance.
- No RJ, o efeito é ainda mais intenso, subindo **de 23% para 73%**.
- Isso sugere que práticas associadas a superhosts - comunicação eficiente, consistência, cancelamentos baixos e boa experiência do usuário - **impactam fortemente os resultados da performance**.

---

### **2. Portfólios maiores aparecem de maneira distinta nos grupos de performance**

#### **NYC**
- O grupo *base* tem média extremamente alta (≈ 80 listings por host), muito maior que os demais.
- Isso ocorre porque existem alguns hosts com **portfólios gigantescos (até 1148 anúncios)**, o que puxa a média para cima.
- Contudo, os *top performers* têm portfólios menores em média (≈ 9 listings), mas mais focados e consistentes.
- A mediana reforça isso:
  - Base: **1 listing**
  - High_perf: **3 listings**
  - Top performer: **2 listings**

**Em NYC, performance alta está mais associada à qualidade do portfólio e não à quantidade.**

#### **Rio de Janeiro**
- Os valores são mais moderados e coerentes com o tamanho do mercado.
- Médias:
  - Base: 9.69
  - High_perf: 12.16
  - Top performer: 9.00
- A mediana de todos é **2 listings**, indicando que hosts casuais e semi-profissionais dominam o mercado.

**No RJ, portfólios médios (2–9 anúncios) tendem a performar melhor que portfólios extremamente grandes.**

---

### **3. Distribuição por categorias de profissionalização**

#### **NYC**
- Base:
  - 14.8k casuais  
  - 5.1k semi-profissionais  
  - 8.4k profissionais  
- Top performers:
  - 1.48k casuais  
  - 1.04k semi-profissionais  
  - 713 profissionais  

O padrão mostra que:

- **Hosts casuais** aparecem bastante em todos os grupos - o mercado é muito heterogêneo.
- **Profissionais** são relevantes, mas não dominantes entre top performers.
- A categoria que mais cresce proporcionalmente entre top performers é **semi-profissional**.

Em NYC, o melhor desempenho parece estar associado a um **portfólio médio** (2–10 anúncios), não aos extremos.

#### **Rio de Janeiro**
- Base:
  - 15.1k casuais  
  - 9.5k semi  
  - 7.9k profissionais  
- Top performers:
  - 1.56k casuais  
  - 1.51k semi  
  - 1.22k profissionais  

Aqui, o padrão é mais uniforme:

- Todos os tipos de hosts aparecem em todos os grupos.
- Entretanto, os grupos *high_perf* e *top_performer* têm **proporção bem maior de semi-profissionais e profissionais** que o grupo base.
- O aumento é mais forte do que em NYC.

No RJ, **profissionalização** tem peso mais consistente na performance.

---

### **4. Comparação geral RJ × NYC**

| Critério | NYC | Rio de Janeiro |
|---------|-----|----------------|
| Superhosts | mercado mais distribuído | superhosts têm impacto mais forte |
| Portfólios gigantes | mais comuns (até 1.148) | raros (máx 232) |
| Top performers | mais associados a foco (portfólio médio) | mais associados à profissionalização |
| Homogeneidade | mercado muito heterogêneo | mercado mais concentrado |

---

### **Conclusão da Análise dos Hosts**

A análise mostra que:

- **Ser superhost é um dos fatores mais fortemente associados à alta performance**, em ambas as cidades.
- Em NYC, top performers tendem a vir de hosts com portfólios mais focados (2–10 anúncios).
- Em RJ, o padrão é mais linear: **profissionalização (2+ anúncios) aumenta a chance de boa performance**.
- Portfólios gigantes não necessariamente têm boa performance - em muitos casos, a qualidade do atendimento se dilui.
- Os resultados reforçam a importância de:
  - consistência operacional,  
  - experiência do host,  
  - e foco no gerenciamento dos anúncios.

# Conclusões da Pergunta 1 - O que diferencia anúncios de alta performance?

A Pergunta 1 buscou identificar **quais características diferenciam os anúncios de alta performance** no Airbnb, a partir da métrica de ocupação estimada nos últimos 365 dias (`estimated_occupancy_l365d`), com comparação entre **Rio de Janeiro** e **New York City**.

A análise foi estruturada em três etapas principais:

1. Definição de uma métrica de performance baseada em percentis por cidade.  
2. Segmentação dos anúncios em três grupos: **Base**, **High Performance** e **Top Performers**.  
3. Comparação desses grupos em termos de:
   - tipo de imóvel (room_type),
   - capacidade (accommodates, bedrooms, beds),
   - preço,
   - reviews,
   - comportamento e perfil dos hosts.

A seguir, as principais conclusões.

---

## 1. Estrutura de performance: poucos anúncios concentram a maior parte da ocupação

- As distribuições de `estimated_occupancy_l365d` em ambas as cidades são **fortemente assimétricas**:
  - a mediana é baixa (0 em NYC, 18 no RJ),
  - o Top 25% (p75) e o Top 10% (p90) têm ocupação muito superior ao restante da amostra.
- Com isso, definimos, por cidade:
  - **Alta performance** → acima do p75;
  - **Top performers** → acima do p90.

**Conclusão:**  
A maior parte dos anúncios tem baixa ou média ocupação, enquanto uma minoria (Top 10–25%) concentra boa parte da demanda. A Pergunta 1 foca justamente em entender o que caracteriza essa minoria.

---

## 2. Tipo de imóvel: “Entire home/apt” domina a alta performance

### New York City
- **Entire home/apt** é mais frequente no grupo **High Performance**.
- **Private rooms** têm participação relevante entre os **Top Performers**, mostrando que, em NYC, não apenas imóveis inteiros, mas também unidades privativas bem posicionadas podem atingir alta performance.
- Shared room e Hotel room têm participação residual nos grupos superiores.

### Rio de Janeiro
- **Entire home/apt** domina de forma quase absoluta os grupos **High Performance** e **Top Performer**.
- Private rooms perdem espaço à medida que a performance aumenta.
- Shared e Hotel rooms são praticamente irrelevantes entre os melhores anúncios.

**Conclusão:**  
- Em ambas as cidades, **anúncios de imóvel inteiro** são os principais candidatos à alta performance.  
- No Rio, essa relação é mais rígida (quase todos os top performers são Entire home/apt).  
- Em NYC, há mais espaço para **private rooms bem avaliados** entrarem no grupo de elite.

---

## 3. Capacidade: imóveis compactos e de porte médio performam melhor

### New York City
- Médias de `accommodates` ficam em torno de **2–3 hóspedes** em todos os grupos.
- Top performers têm leve aumento em camas, mas continuam majoritariamente com **1 quarto** e capacidade pequena.
- A maior parte da alta performance está em unidades para **1–2 hóspedes**, com participação secundária de imóveis maiores.

### Rio de Janeiro
- A mediana de capacidade nos três grupos é de **4 hóspedes**.
- À medida que a performance aumenta, a **capacidade média tende a cair levemente**:
  - top performers são, em média, um pouco menores que os anúncios base.
- Faixas mais fortes entre high perf e top performers:
  - **3–4 hóspedes**, com menor participação de imóveis muito grandes (7+ hóspedes).

**Conclusão:**  
- Em **NYC**, a alta performance é dominada por **unidades compactas** (1–2 hóspedes).  
- No **Rio**, a alta performance se concentra em **imóveis de porte médio** (3–4 hóspedes).  
- Imóveis muito grandes, em geral, **não dominam** os grupos de melhor desempenho.

---

## 4. Preço: faixas intermediárias concentram os anúncios de melhor performance

- Em ambos os mercados, os grupos **High Performance** e **Top Performer** tendem a se concentrar em **faixas intermediárias de preço**, com:
  - menos dispersão,
  - menos extremos (nem os mais baratos, nem os mais caros do mercado).
- Em NYC, os top performers se alinham majoritariamente na faixa **USD 100–300**.
- No Rio, os melhores desempenhos se concentram entre **BRL 100–500**, enquanto o grupo base é mais espalhado, incluindo muitos imóveis muito baratos e muitos imóveis muito caros.

**Conclusão:**  
Anúncios de alta performance têm, em geral, **preço estrategicamente posicionado em faixas médias**, evitando tanto o “fundo do poço” quanto o nicho de luxo extremo.  
Performance alta parece estar ligada a **preço competitivo**, alinhado ao segmento alvo de cada cidade.

---

## 5. Reviews: fluxo e recência importam mais que a nota em si

- As notas (`review_scores_rating`) são, em média, **altas em todos os grupos**, o que limita poder discriminativo:
  - diferenças existem (grupos de alta performance têm notas ligeiramente superiores),
  - mas o rating por si só não explica a performance.

- Já os indicadores de **dinâmica de reviews** se mostram fortemente associados à alta performance:
  - **reviews_per_month** é muito maior nos grupos High Performance e Top Performer;
  - **review_recency_days** é muito menor entre anúncios de alta performance (reviews recentes, atividade constante).

**Conclusão:**  
Não é a “nota bonita” que diferencia os anúncios de alta performance - e sim o **fluxo constante de reservas e avaliações recentes**.  
Ou seja, **demanda ativa** é o verdadeiro motor por trás das métricas de review dos top performers.

---

## 6. Comportamento e perfil dos hosts: superhosts e profissionalização com foco

A análise de host trouxe alguns padrões consistentes:

### Superhosts

- A proporção de superhosts cresce fortemente do grupo Base para High Perf e Top Performer, em **ambas as cidades**.
  - Em NYC: de ~10% (base) para >60% (top performers).
  - No RJ: de ~23% (base) para >70% (top performers).

**Leitura:**  
Ser superhost está **fortemente associado** à alta performance, refletindo boa experiência do hóspede, baixa taxa de cancelamento, boa comunicação e consistência operacional.

### Tamanho do portfólio

- Em **NYC**, existem hosts com portfólios gigantescos (centenas de listings), o que distorce médias no grupo base.  
  Já entre top performers, a mediana converge para hosts com **2–3 anúncios**, sugerindo foco maior na qualidade do portfólio que na quantidade.
- No **Rio**, portfólios são menores e mais equilibrados, com mediana ~2 anúncios para todos os grupos.  
  A alta performance é mais frequente entre **semi-profissionais (2–5 anúncios)** e **profissionais (6+ anúncios)**, mas sem concentração em extremos.

**Conclusão:**  
- Em ambos os mercados, **hosts mais experientes e organizados** (superhosts e com portfólios minimamente estruturados) aparecem com frequência desproporcional entre os anúncios de alta performance.  
- Portfólios gigantes não garantem boa performance; em muitos casos, a performance máxima vem de **portfólios médios, mais bem geridos**.

---

## Conclusão Geral da Pergunta 1

Integrando todas as dimensões analisadas, os anúncios de alta performance no Airbnb, em Rio de Janeiro e New York City, tendem a compartilhar o seguinte perfil:

1. **Tipo de imóvel**
   - Predominância de **Entire home/apt**, especialmente no Rio.
   - Em NYC, há espaço também para private rooms bem posicionados.

2. **Capacidade**
   - NYC: unidades compactas (1–2 hóspedes) predominam na alta performance.
   - RJ: unidades de porte médio (3–4 hóspedes) concentram o melhor desempenho.

3. **Preço**
   - Faixas intermediárias de preço concentram a maior parte dos high performers.
   - Preços muito baixos ou muito altos são mais comuns no grupo base.

4. **Reviews**
   - Alta performance está ligada a **fluxo contínuo de reviews recentes**, não apenas à nota média.

5. **Hosts**
   - Superhosts são desproporcionalmente representados entre top performers.
   - Portfólios médios, bem gerenciados, tendem a performar melhor do que portfólios dispersos e gigantescos.

Em resumo, **anúncios de alta performance** não são apenas “bons imóveis em bons lugares”, mas sim o resultado de uma combinação de:

- estrutura adequada (imóvel inteiro, capacidade coerente com a demanda local),
- precificação alinhada ao mercado,
- demanda ativa e constante,
- e gestão profissional do anúncio pelo host.

Esses achados preparam o terreno para as próximas perguntas do MVP, que aprofundam a análise em **preço** (Pergunta 2) e **localização** (Pergunta 3).


# Pergunta 2 - Quais fatores influenciam o preço da diária e a receita estimada?

### Objetivo da análise

Investigar os elementos que determinam o **preço da diária (`price_numeric`)** e a **receita estimada anual (`estimated_revenue_l365d`)** dos anúncios.  
O objetivo é compreender como diferentes características - do imóvel, do host e do comportamento de demanda - influenciam diretamente a precificação e a rentabilidade no Airbnb.

Serão considerados fatores como:

- tipo de imóvel (`room_type`, `property_type`);
- capacidade (`accommodates`, `bedrooms`, `beds`);
- reputação e engajamento via reviews;
- profissionalização do host (`host_is_superhost_bool`, tamanho do portfólio);
- influência da ocupação estimada sobre a receita anual.

A meta é identificar **quais atributos exercem maior influência sobre o preço e quais deles estão associados às receitas mais elevadas**, permitindo comparar Rio de Janeiro e NYC com clareza.

---

### Estratégia da análise

1. Descrever a distribuição dos preços entre Rio e NYC:
   - valores médios, medianas e dispersão;
   - variação por grupo de performance.

2. Investigar os fatores que explicam variações de preço:
   - tipo de imóvel e capacidade,
   - características internas (bedrooms, beds),
   - reviews e ratings,
   - superhost e profissionalização,
   - ocupação estimada.

3. Relacionar **preço** com **receita anual estimada**, avaliando:
   - elasticidade entre preço e receita,
   - impacto da ocupação,
   - segmentos com maior potencial de rentabilidade.

4. Comparar padrões entre as duas cidades, destacando diferenças estruturais no mercado e no comportamento de precificação.

5. Extrair os fatores mais determinantes do preço e da receita, respondendo objetivamente à pergunta de negócio.


### Análise - Relação entre Tipo de Imóvel e Preço da Diária

Nesta primeira etapa da Pergunta 2, investigamos como o **tipo de quarto** influencia o preço da diária (`price_numeric`).  
A análise é separada por cidade, pois Rio de Janeiro e NYC possuem moedas, padrões de demanda e políticas de precificação distintas.

Serão considerados:

- **room_type**  
  - Entire home/apt  
  - Private room  
  - Shared room  
  - Hotel room  

O objetivo é identificar:

- quais tipos de quarto possuem preços consistentemente mais altos;
- se a diferença de preço entre room types é relevante dentro de cada cidade;
- como a categoria do quarto afeta a precificação, independentemente de performance;
- se há padrões claros de precificação - tipos que naturalmente custam mais.

Essa análise estabelece a base para as próximas, permitindo entender se o preço é influenciado mais pela **estrutura do que está sendo alugado**, pela **capacidade**, pelas **reviews**, ou pelos **fatores do host**.


In [0]:
# Cópia da base
df_type_price = df_listings.copy()

# Mantém apenas anúncios com preço válido
df_type_price = df_type_price[df_type_price["price_numeric"].notna()].copy()

# 1) Preço por city x room_type
summary_room_type_price = (
    df_type_price
    .groupby(["city", "room_type"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)


summary_room_type_price


### Interpretação dos Resultados - Relação entre Room Type e Preço da Diária

A análise do `room_type` mostra padrões muito claros de precificação em ambas as cidades.  
Os valores absolutos não são comparáveis entre Rio (BRL) e NYC (USD), mas **as relações internas** revelam como cada tipo de imóvel influencia fortemente o preço.

---

### **1. Entire home/apt é consistentemente o tipo mais caro e segue uma lógica clara de mercado**

- **NYC**: mediana de **210 USD**  
- **RJ**: mediana de **350 BRL**

Isso reflete que:
- oferecer o imóvel inteiro tem maior valor percebido,
- indica maior privacidade, conforto e espaço,
- atrai hóspedes com maior orçamento.

Além disso, o intervalo interquartil (P25–P75) é o mais amplo nas duas cidades, mostrando que essa categoria acomoda desde imóveis simples até opções premium.

---

### **2. Private rooms representam o segmento intermediário e mais acessível dos dois mercados**

- Mediana NYC: **86 USD**  
- Mediana RJ: **205 BRL**

Padrões observados:
- São mais baratos que entire homes, mas representam uma oferta de boa relação custo-benefício.
- A dispersão é muito maior em NYC, com preços indo de 58 (P25) a 148 (P75), enquanto no RJ há menos variação relativa.
- Isso sugere um mercado mais homogêneo no RJ para private rooms e mais segmentado em NYC.

---

### **3. Shared rooms são a base da pirâmide de preços**

- Mediana NYC: **67 USD**  
- Mediana RJ: **105 BRL**

Indicando:
- oferta destinada a mochileiros, viajantes econômicos e estadias curtas;
- variação muito pequena no RJ (74–149 BRL), o que reforça baixo grau de diferenciação nesse segmento.

---

### **4. Hotel room em NYC é um outlier extremo**

NYC mostra:
- média de **36.088 USD**
- mediana de **40.000 USD**

Esses valores são incompatíveis com o mercado real → muito provavelmente anúncios “fakes”, testes ou **problemas na origem dos dados do InsideAirbnb**, que são conhecidos por conter outliers artificiais na categoria “Hotel room”.

No RJ, os valores são normais e coerentes:
- mediana de **470 BRL**
- P25–P75 entre **263–670 BRL**

Para a análise, o ideal será tratar NYC–Hotel room como **outlier removido** nas próximas etapas, pois pode distorcer correlações de preço.

---

### **5. Relação geral dos room types com o preço**

Em ambos os mercados, a hierarquia é idêntica:

1. **Entire home/apt - mais caro**  
2. **Hotel room - depende da cidade (problemático em NYC)**  
3. **Private room - segmento intermediário**  
4. **Shared room - mais barato**

**Ponto-chave:**  
O preço está fortemente associado ao nível de privacidade e ao tipo de estrutura oferecida.

---

### **Conclusão da Análise do Room Type**

- O tipo de imóvel é um dos fatores mais fortes na determinação do preço.  
- A lógica de preço é consistente entre as duas cidades, independentemente da moeda.  
- Entire homes são o padrão premium; shared rooms, o segmento econômico.  
- NYC apresenta uma anomalia relevante na categoria “Hotel room”, indicando necessidade de limpeza antes de análises correlacionais.  
- Essa análise estabelece a base para entender o impacto da **capacidade (accommodates, bedrooms, beds)**, que investigaremos na próxima etapa.



### Nota complementar - Outlier na categoria *Hotel room* em NYC

Durante a análise, foi identificado um comportamento anômalo na categoria **Hotel room** de New York City, com valores extremamente elevados (mediana acima de 40.000 USD).  
Esse padrão não é compatível com o mercado real e provavelmente se origina de:

- erros na origem dos dados do InsideAirbnb,
- anúncios inativos ou placeholders,
- valores artificiais inseridos por hosts ou pelo próprio scraping.

Como o número de anúncios nessa categoria é muito pequeno (234 registros) e os valores distorcem significativamente as métricas de NYC, o ideal é **tratar essa categoria como outlier** nas análises subsequentes de preço.

Essa decisão não afeta as conclusões sobre *room_type*, que permanecem válidas para as categorias principais:
- Entire home/apt  
- Private room  
- Shared room



### Análise - Relação entre Capacidade do Imóvel e Preço da Diária

Nesta etapa da Pergunta 2, investigamos como características internas do imóvel - em especial a **capacidade** - influenciam o preço da diária (`price_numeric`).  
A análise é feita separadamente para Rio de Janeiro e New York City, pois os mercados operam em moedas distintas e possuem dinâmicas próprias de demanda.

Serão avaliados três atributos centrais:

- **accommodates** - número total de hóspedes suportados;
- **bedrooms** - número de quartos;
- **beds** - número de camas disponíveis.

O objetivo é entender:

- se o preço cresce proporcionalmente ao aumento da capacidade;
- quais atributos (accommodates, bedrooms ou beds) têm relação mais forte com o preço;
- se existe um “ponto de saturação” onde aumentar a capacidade deixa de aumentar o valor cobrado;
- como essa relação varia entre RJ e NYC.

Essa análise complementa diretamente os achados sobre *room_type* e ajuda a explicar a formação de preço dentro de cada cidade.


In [0]:
 # Cópia da base
df_capacity = df_listings.copy()

# Mantém apenas anúncios com preço válido
df_capacity = df_capacity[df_capacity["price_numeric"].notna()].copy()

# Preço por accommodates
summary_accommodates_price = (
    df_capacity
    .groupby(["city", "accommodates"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_accommodates_price

In [0]:
# Preço por bedrooms
summary_bedrooms_price = (
    df_capacity
    .groupby(["city", "bedrooms"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_bedrooms_price


In [0]:

# Preço por beds
summary_beds_price = (
    df_capacity
    .groupby(["city", "beds"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_beds_price


### Interpretação dos Resultados - Relação entre Capacidade e Preço da Diária

A análise de capacidade - considerando `accommodates`, `bedrooms` e `beds` - revela padrões claros, mas também evidencia pontos de saturação e distorções que se tornam decisivos para entender como o preço é formado em cada cidade.

Para ambas as cidades, analisamos as relações **internas** (dentro de cada moeda) e não comparamos valores absolutos entre NY e RJ.

---

## **Relação entre accommodates e preço: efeito claro, porém não linear**

### New York City
- Preços crescem conforme aumenta a capacidade até aproximadamente **4–6 hóspedes**.
- Entre `accommodates` 1–6:
  - as medianas sobem de 64 USD → 297 USD.
  - esse comportamento é consistente.
- A partir de `accommodates` 7+, surgem **outliers severos**:
  - especialmente em 8, 12 e 16 hóspedes, com médias altas (3675, 9136 e 3989 USD).
  - Mesmo com medianas moderadas, o “preço médio” explode por causa de poucos anúncios extremamente caros.

**Em NYC, o crescimento do preço deixa de ser estável após 6 hóspedes.**  
Há um segmento de ultra-luxo e mansões que distorce a média.

### Rio de Janeiro
- Comportamento semelhante, porém mais suave.
- Preços sobem até aproximadamente **6–7 hóspedes**, de forma consistente.
- Acima disso, o mercado se segmenta e surgem imóveis muito caros (faixa de luxo na Zona Sul).
- Nos extremos (10+ hóspedes), o mesmo fenômeno de NYC aparece:  
  **média explosiva e mediana mais estável**.

**O efeito da capacidade existe e é forte, mas fica cada vez mais disperso quanto maior o imóvel.**

---

## **Relação entre bedrooms e preço: a variável mais estável e mais informativa**

Em ambas as cidades, `bedrooms` apresenta:

- comportamento mais previsível,
- variação menor no centro da distribuição,
- menor impacto de outliers em comparação a `accommodates`.

### New York City
- Crescimento mais limpo até 3 quartos.
- A partir de 4+, volta o efeito das mansões e imóveis atípicos.
- Mediana sobe de 170 USD → 260 USD → 260–400 USD para 3–5 quartos.
- Para 8+ quartos, são claramente imóveis excepcionais (1–3 anúncios por categoria).

### Rio de Janeiro
- Relação muito nítida até 4 quartos:
  - 223 BRL → 250 BRL → 463 BRL → 743 BRL.
- A partir de 5 quartos, preços entram na faixa de imóveis de altíssimo padrão.
- Valores medianos acima de 3.000, 7.000 e até 14.000 BRL evidenciam esse nicho de luxo extremo.

**Bedrooms é o melhor preditor de preço entre as três variáveis de capacidade.**

---

## **Relação entre beds e preço: comportamento intermediário, porém mais ruidoso**

Beds tem relação com preço, mas:

- sofre mais com outliers,
- depende muito do layout do imóvel,
- é menos consistente do que bedrooms.

### New York City
- Crescimento coerente entre 1–4 camas.
- A partir de 8 camas, surgem novamente valores inflados devido a propriedades raras e luxo.
- Medianas continuam estáveis, mesmo quando as médias disparam.

### Rio de Janeiro
- Crescimento claro até 4 camas.
- A partir daí, forte dispersão:
  - medianas entre 500 e 1.500 BRL,
  - mas médias variando de 1.600 a 10.000+ BRL.
- Novamente, extremos dominados por imóveis de luxo.

 **Beds ajuda a entender o preço, mas não é tão determinante quanto bedrooms.**

---

## **Síntese da Análise de Capacidade**

- **Bedrooms** → variável mais forte e mais estável na explicação do preço.  
- **Accommodates** → boa indicação até 6 hóspedes; depois disso, ruído por imóveis de luxo.  
- **Beds** → útil, mas menos confiável por variações de layout e outliers.  

Em ambas as cidades, a capacidade influencia o preço, mas:

### - O impacto é forte em imóveis pequenos e médios.  
### - O impacto se dilui em imóveis grandes (7+ hóspedes / 4+ quartos).  
### - Outliers dominam o topo da distribuição em ambos os mercados.  
### - O mercado de luxo distorce médias - medianas são a medida mais confiável.

---

### **Conclusão da Análise de Capacidade**

A capacidade é um dos fatores mais importantes para a formação de preço, mas não de forma linear.  
Os resultados mostram:

- crescimento estável até faixas intermediárias;
- saturação e dispersão nos segmentos maiores;
- bedrooms como o melhor indicador estrutural;
- presença marcada de imóveis de luxo que precisam ser interpretados com cautela.


### Análise - Relação entre Reputação (Reviews) e Preço da Diária

Nesta etapa da Pergunta 2, avaliamos de que forma variáveis de reputação e engajamento do anúncio influenciam o preço da diária (`price_numeric`).  
A análise é feita separadamente para Rio de Janeiro e New York City, respeitando as diferenças estruturais entre os mercados.

As variáveis consideradas são:

- `review_scores_rating` - nota média do anúncio;
- `reviews_per_month` - frequência recente de avaliações, indicadora de demanda;
- `review_recency_days` - quantidade de dias desde a última avaliação.

O objetivo é identificar:

- se anúncios com notas mais altas tendem a praticar preços maiores;
- se maior engajamento (reviews por mês) está associado a preços mais elevados;
- se anúncios com reviews recentes apresentam comportamento distinto de precificação;
- se o efeito dessas variáveis é forte, moderado ou secundário em relação a fatores estruturais do imóvel.

Essa análise complementa os fatores anteriores (room_type e capacidade) e ajuda a determinar se o preço é influenciado não apenas pela estrutura física do anúncio, mas também pela percepção de qualidade e pela dinâmica recente de demanda.


In [0]:
# Cópia da base
df_reviews = df_listings.copy()

# Mantém apenas anúncios com preço válido
df_reviews = df_reviews[df_reviews["price_numeric"].notna()].copy()

# Buckets de rating
rating_bins = [0, 4.0, 4.5, 4.7, 4.85, 5.01]
rating_labels = [
    "<= 4.0",
    "4.01 - 4.50",
    "4.51 - 4.70",
    "4.71 - 4.85",
    ">= 4.86"
]

df_reviews["rating_bucket"] = pd.cut(
    df_reviews["review_scores_rating"],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True
)

summary_rating_price = (
    df_reviews
    .groupby(["city", "rating_bucket"])
    .agg(
        count_listings=("listing_id", "count"),
        rating_mean=("review_scores_rating", "mean"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)



summary_rating_price

In [0]:
# Buckets de reviews/mês
rpm_bins = [-0.01, 0, 0.5, 1, 2, 5, df_reviews["reviews_per_month"].max()]
rpm_labels = ["0", "0.01 - 0.50", "0.51 - 1.00", "1.01 - 2.00", "2.01 - 5.00", "> 5.00"]

df_reviews["reviews_per_month_bucket"] = pd.cut(
    df_reviews["reviews_per_month"].fillna(0),
    bins=rpm_bins,
    labels=rpm_labels,
    include_lowest=True
)

summary_rpm_price = (
    df_reviews
    .groupby(["city", "reviews_per_month_bucket"])
    .agg(
        count_listings=("listing_id", "count"),
        rpm_mean=("reviews_per_month", "mean"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_rpm_price

In [0]:
# Buckets de recência do review
def recency_bucket_func(x):
    if pd.isna(x):
        return "sem_reviews"
    if x <= 30:
        return "0 - 30 dias"
    elif x <= 90:
        return "31 - 90 dias"
    elif x <= 180:
        return "91 - 180 dias"
    elif x <= 365:
        return "181 - 365 dias"
    else:
        return "> 365 dias"

df_reviews["recency_bucket"] = df_reviews["review_recency_days"].apply(recency_bucket_func)

summary_recency_price = (
    df_reviews
    .groupby(["city", "recency_bucket"])
    .agg(
        count_listings=("listing_id", "count"),
        recency_mean=("review_recency_days", "mean"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_recency_price

### Interpretação dos Resultados - Relação entre Reputação (Reviews) e Preço da Diária

Os resultados indicam que reputação e engajamento têm relação com o preço, mas essa relação é mais sutil e bem menos determinante do que fatores estruturais do imóvel (tipo, capacidade, localização). Em vários casos, o efeito é moderado e aparece mais na dispersão do que na mediana.

---

### 1. Relação entre nota média (rating) e preço

Em ambas as cidades, a maioria dos anúncios apresenta notas altas (acima de 4,5), o que reduz a variabilidade entre faixas. Ainda assim, alguns padrões aparecem.

- Em New York City:
  - imóveis com rating até 4.0 têm preço médio mais alto (585.72) do que faixas intermediárias, mas a mediana é baixa (130).  
    Isso indica presença de outliers de luxo com notas baixas (ou poucas reviews), distorcendo a média.
  - nas faixas de 4.51 a 4.70 e 4.71 a 4.85, as medianas ficam estáveis, entre 140 e 144 USD.
  - a faixa de rating mais alta (≥ 4.86) apresenta preço médio maior (380.43), porém mediana de 155 USD, bastante próxima das demais faixas altas.

- No Rio de Janeiro:
  - há comportamento semelhante: faixa ≤ 4.0 com média mais alta (636.34), mas mediana de 306 BRL.
  - as faixas intermediárias (4.51–4.70 e 4.71–4.85) apresentam preços médios e medianas mais baixos do que a faixa extrema de ≥ 4.86.
  - anúncios com rating ≥ 4.86 têm preço médio maior (574.71) e mediana de 309 BRL, um pouco acima das faixas intermediárias.

Em resumo:

- anúncios com ratings muito baixos não são necessariamente mais baratos; em muitos casos, tratam-se de imóveis de valor alto com pouca base de avaliação;
- nas duas cidades, existe um leve aumento de preço nas faixas de rating mais alto, mas o efeito é moderado, e as medianas são próximas;
- a reputação parece mais associada à confiança e à demanda do que a um aumento agressivo de preço.

---

### 2. Relação entre reviews por mês (engajamento) e preço

Aqui avaliamos se anúncios com maior giro de reviews praticam preços diferentes.

- Em New York City:
  - anúncios sem reviews recentes (bucket 0) apresentam preço médio muito elevado (1455.37), mas mediana de 185 USD.  
    A disparidade média × mediana indica forte presença de imóveis de luxo com baixa frequência de avaliação.
  - nas faixas de 0.01 a 2 reviews/mês, o preço médio oscila entre 245 e 522 USD, com medianas concentradas entre 138 e 172 USD.
  - na faixa de maior engajamento (> 5 reviews/mês), o preço médio sobe para 375.55, com mediana de 214.5 USD.

- No Rio de Janeiro:
  - o bucket 0 também concentra outliers: preço médio de 1421.91 e mediana de 485 BRL, bem acima das demais faixas.
  - conforme o engajamento aumenta (0.51–1.00, 1.01–2.00, 2.01–5.00), o preço médio tende a cair gradualmente (457.86 → 405.60 → 344.11), com medianas caindo de 285 para 255 BRL.
  - imóveis com engajamento muito alto (> 5 reviews/mês) apresentam preços mais baixos em média (243.57; mediana 190), sugerindo que alta rotatividade pode estar associada a faixas de preço mais acessíveis.

Em termos de padrão geral:

- anúncios sem reviews ou com reviews muito espaçados são os que mais concentram outliers de preço;
- engajamento moderado está associado a uma faixa de preço intermediária, coerente com boa demanda;
- em RJ, há evidência de que imóveis muito bem avaliados e acessíveis conseguem alto volume de reservas, o que puxa o preço médio desses buckets para baixo.

---

### 3. Relação entre recência do review e preço

A variável de recência captura quão “atual” é o histórico de reviews do anúncio, funcionando como um proxy de atividade recente.

- Em **New York City**:
  - Anúncios com reviews recentes (31–90 dias) apresentam preço médio de aproximadamente **336.65 USD** e mediana de **170 USD**, valores próximos aos dos grupos de 91–180 dias e 181–365 dias.
  - O grupo de **91–180 dias** apresenta o maior preço médio (**469.00 USD**), indicando que anúncios relativamente ativos podem sustentar preços mais elevados.
  - A categoria **"> 365 dias"** apresenta preço médio menor (**279.60 USD**) e mediana de **149 USD**, sugerindo que anúncios com longo período sem atividade tendem a se posicionar em faixas mais moderadas.
  - O grupo **sem reviews** volta a concentrar preços médios muito altos (**1455.37 USD**), efeito causado por poucos imóveis de luxo ainda não avaliados.

- No **Rio de Janeiro**:
  - Anúncios com reviews recentes (31–90 dias) apresentam preço médio de **355.26 BRL** e mediana de **258 BRL**.
  - As faixas de **91–180 dias** e **181–365 dias** apresentam elevação no preço médio, alcançando **428.13 BRL** e **562.50 BRL**, respectivamente.
  - O bucket **"> 365 dias"** registra preço médio elevado (**1162.29 BRL**) e mediana de **414 BRL**, indicando que parte dos imóveis mais caros possui histórico antigo de reviews.
  - Assim como em NYC, o grupo **sem reviews** concentra valores elevados (média de **1421.91 BRL**), influenciado por anúncios de alto padrão ainda sem avaliações.

De forma geral, a recência do review não apresenta uma relação linear direta com o preço, mas indica que anúncios mais ativos tendem a se concentrar em faixas intermediárias, enquanto imóveis caros aparecem com maior frequência tanto em faixas de baixa atividade quanto entre anúncios ainda não avaliados.

Os resultados sugerem:

- em RJ, imóveis mais caros tendem a ter histórico de reviews mais antigo e menos giro recente;
- em ambas as cidades, imóveis sem reviews formam um grupo heterogêneo, com forte presença de anúncios de alto valor que podem ser novos, específicos ou pouco demandados.

---

### Conclusão da Análise de Reputação e Preço

A reputação impacta o preço, mas não de forma tão direta quanto a estrutura física do imóvel:

- rating mais alto está associado a preços um pouco maiores, mas a diferença entre faixas intermediárias e extremas é moderada;
- maior engajamento em reviews por mês não implica necessariamente preços mais altos; em muitos casos, imóveis mais acessíveis obtêm maior volume de avaliações;
- recência de reviews parece ter relação mais forte com o perfil de demanda do que com o nível absoluto de preço;
- grupos sem reviews ou com baixa atividade concentram uma mistura de imóveis de luxo e anúncios pouco competitivos, o que distorce as médias.

De forma geral, reputação e engajamento funcionam como fatores de ajuste fino na precificação: ajudam a sustentar preços dentro de uma faixa alta ou média, mas não são os principais determinantes do valor da diária quando comparados a tipo de imóvel, capacidade e localização.

Essa análise reforça a visão de que variáveis de reviews são mais relevantes para explicar demanda e ocupação do que para justificar grandes diferenças de preço por si só.


### Análise - Relação entre Profissionalização do Host e Preço da Diária

Nesta etapa da Pergunta 2, investigamos se características associadas à profissionalização do anfitrião influenciam a precificação dos anúncios no Airbnb. A análise é feita separadamente para Rio de Janeiro e New York City.

As variáveis utilizadas são:

- `host_is_superhost_bool` - indica se o host possui o selo Superhost;
- `calculated_host_listings_count` - número de anúncios sob o mesmo host (medida de escala e profissionalização).

Os objetivos desta análise são:

- verificar se Superhosts cobram preços mais elevados;
- entender como o tamanho do portfólio do host afeta o preço médio e mediano dos anúncios;
- identificar se há diferenças entre hosts ocasionais (1 anúncio), semi-profissionais (2–5 anúncios) e profissionais (6+ anúncios);
- avaliar se a profissionalização do host é um fator relevante na formação de preço em cada cidade.

Essa análise complementa os resultados anteriores sobre tipo de imóvel, capacidade e reputação, permitindo avaliar o papel do comportamento e da estrutura operacional do host no modelo final de precificação.


In [0]:
# Cópia da base
df_prof = df_listings.copy()

# Mantém apenas anúncios com preço válido
df_prof = df_prof[df_prof["price_numeric"].notna()].copy()

# Construindo o tamanho de portfólio por host (quantos anúncios cada host possui na cidade)
df_host_portfolio_price = (
    df_prof
    .groupby(["city", "host_id"])
    .agg(
        host_listings_count=("listing_id", "nunique")
    )
    .reset_index()
)

# Juntando o portfólio na base de anúncios
df_prof = df_prof.merge(
    df_host_portfolio_price,
    on=["city", "host_id"],
    how="left"
)

# 1) Preço por superhost vs não-superhost
summary_superhost_price = (
    df_prof
    .groupby(["city", "host_is_superhost_bool"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)



summary_superhost_price


In [0]:
# 2) Buckets de profissionalização por tamanho de portfólio
def classify_host_portfolio(n):
    if n == 1:
        return "casual"
    elif 2 <= n <= 5:
        return "semi_profissional"
    else:
        return "profissional"

df_prof["host_category"] = df_prof["host_listings_count"].apply(classify_host_portfolio)

summary_host_category_price = (
    df_prof
    .groupby(["city", "host_category"])
    .agg(
        count_listings=("listing_id", "count"),
        portfolio_mean=("host_listings_count", "mean"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_host_category_price

### Interpretação dos Resultados - Profissionalização do Host e Preço da Diária

Os resultados mostram que a profissionalização do host influencia o preço, mas de forma indireta e fortemente afetada por outliers. Em vários casos, o efeito é melhor observado nas medianas do que nas médias.

---

### 1. Superhost versus não-superhost

#### New York City

- Não-superhosts:
  - preço médio: 887.08
  - mediana: 158.0

- Superhosts:
  - preço médio: 232.91
  - mediana: 150.0

A mediana dos dois grupos é muito próxima (150–158), indicando que:

- na maior parte do mercado, Superhosts não cobram significativamente mais do que não-superhosts;
- a diferença brutal na média (887 vs 232) é um sinal claro de que há **não-superhosts com preços extremamente altos**, que distorcem o valor médio.

Ou seja: em NYC, ser Superhost não é um fator de aumento direto de preço; o que existe é uma cauda de imóveis caros sem selo de Superhost que puxam a média dos não-superhosts para cima.

#### Rio de Janeiro

- Não-superhosts:
  - preço médio: 798.80
  - mediana: 349.0

- Superhosts:
  - preço médio: 590.83
  - mediana: 283.0

Aqui o padrão se repete:

- não-superhosts têm média e mediana maiores;
- a diferença de mediana (283 vs 349) sugere que, na prática, Superhosts tendem a praticar preços um pouco menores.

Interpretação:

- Superhosts, em geral, não parecem usar o selo para cobrar mais caro;
- em muitos casos, são justamente os imóveis mais bem operados e mais competitivos em preço, que buscam volume e constância de reservas;
- a combinação “bom produto + preço moderado” tende a gerar o selo Superhost.

Em síntese, os dados indicam que **ser Superhost está mais ligado à estratégia de competitividade e qualidade do que ao aumento direto de preço.**

---

### 2. Tamanho do portfólio (categoria do host) e preço

As categorias foram definidas como:

- casual: 1 anúncio
- semi_profissional: 2 a 5 anúncios
- profissional: 6+ anúncios

#### New York City

- Casual:
  - média: 219.03
  - mediana: 151.0

- Semi-profissional:
  - média: 264.91
  - mediana: 124.0

- Profissional:
  - média: 1406.47
  - mediana: 193.0

O comportamento é o seguinte:

- a média dos profissionais é muito alta (1406.47), mas a mediana é próxima da dos casuais (193 vs 151);
- isso sugere que existe um conjunto relativamente pequeno de anúncios profissionais com preços extremamente altos (luxo, hotéis, operações especiais), que inflaciona a média;
- semi-profissionais têm média maior que casuais, mas mediana menor (124), o que indica maior dispersão e presença de imóveis mais baratos operados por hosts com alguns anúncios.

Conclusão parcial para NYC:

- o tamanho do portfólio não determina de forma simples o nível de preço;
- existe um segmento de profissionais com imóveis de altíssimo valor, mas a maior parte dos anúncios profissionais está em faixa de preço semelhante à dos casuais;
- não há evidência clara de que ser profissional implique sempre em preço mais alto, embora haja uma “cauda premium” muito forte.

#### Rio de Janeiro

- Casual:
  - média: 704.42
  - mediana: 350.0

- Semi-profissional:
  - média: 543.13
  - mediana: 278.0

- Profissional:
  - média: 946.43
  - mediana: 321.0

Aqui o padrão é mais nítido:

- casuais e profissionais têm medianas próximas (350 e 321);
- semi-profissionais têm preços medianos ligeiramente menores (278);
- a média dos profissionais é mais alta (946.43), indicando novamente presença de imóveis de ticket mais alto na categoria profissional;
- casuais apresentam preços medianos mais altos que semi-profissionais, o que pode indicar:
  - proprietários individuais com imóveis bem localizados e caros, mas sem grande portfólio;
  - operações pequenas, porém em nichos premium (por exemplo, imóveis de frente para o mar).

Conclusão parcial para RJ:

- hosts profissionais tendem a participar mais da faixa de preço alta (pela média), mas não formam um grupo completamente separado em termos de mediana;
- semi-profissionais parecem operar mais em faixas de preço intermediárias;
- casuais aparecem tanto em imóveis mais acessíveis quanto em imóveis caros e bem localizados.

---

### Síntese: profissionalização do host e formação de preço

Os dados mostram que:

- não há um efeito simples do tipo “mais profissional = sempre mais caro”;
- Superhosts, em média, **não cobram mais** do que não-superhosts; muitas vezes cobram menos, sugerindo foco em competitividade e ocupação;
- hosts profissionais concentram:
  - uma parte relevante dos imóveis de alto valor,
  - mas mantêm um conjunto grande de anúncios em faixas de preço próximas às dos demais;
- em ambos os mercados, a **dispersão** é muito alta nas categorias profissionais, o que faz com que a média seja pouco confiável como indicador.

Em termos de preço, a profissionalização:

- aparece mais como um fator de segmentação (existe um nicho de luxo concentrado em alguns profissionais),
- do que como um determinante sistemático de preço para todo o portfólio.

Essa análise reforça a ideia de que tipo de imóvel, capacidade e localização são fatores mais diretos na formação do preço, enquanto a profissionalização do host atua principalmente:

- na qualidade da operação,
- na consistência da oferta,
- e na presença em nichos específicos de mercado (particularmente o luxo).


### Análise - Relação entre Ocupação Estimada e Receita Anual

Nesta última etapa da Pergunta 2, investigamos como a ocupação estimada dos anúncios influencia a receita anual.  
As variáveis utilizadas são:

- `estimated_occupancy_l365d` - percentual estimado de ocupação nos últimos 365 dias;
- `estimated_revenue_l365d` - receita anual estimada pelo Inside Airbnb (construída a partir de ocupação × preço × disponibilidade).

O objetivo é identificar:

- se a ocupação é um forte preditor da receita anual (e até que ponto);
- como a receita varia entre diferentes faixas de ocupação;
- se existem faixas de ocupação onde o retorno cresce rapidamente ou se estabiliza;
- diferenças entre Rio de Janeiro e New York City nos padrões de retorno;
- se há grupos de imóveis que conseguem alta receita com ocupação menor (indicando preços mais altos).

A análise é feita separadamente por cidade, considerando que a receita é calculada em moedas distintas.  
O foco está na relação entre as duas variáveis, não na comparação direta dos valores absolutos entre os mercados.


In [0]:
# Cópia da base
df_occ = df_listings.copy()

# Mantém apenas anúncios com ocupação e receita efetivamente válidas (> 0)
df_occ = df_occ[
    (df_occ["estimated_occupancy_l365d"] > 0)
    & (df_occ["estimated_revenue_l365d"] > 0)
].copy()

# Buckets de ocupação em dias (0 a 365)
occ_bins = [0, 90, 180, 270, 366]
occ_labels = [
    "0 - 90 dias",
    "91 - 180 dias",
    "181 - 270 dias",
    "271 - 365 dias"
]

df_occ["occupancy_bucket"] = pd.cut(
    df_occ["estimated_occupancy_l365d"],
    bins=occ_bins,
    labels=occ_labels,
    include_lowest=True
)

# Resumo estatístico por cidade e faixa de ocupação
summary_occupancy_revenue = (
    df_occ
    .groupby(["city", "occupancy_bucket"])
    .agg(
        count_listings=("listing_id", "count"),
        occupancy_mean=("estimated_occupancy_l365d", "mean"),
        revenue_mean=("estimated_revenue_l365d", "mean"),
        revenue_median=("estimated_revenue_l365d", "median"),
        revenue_p25=("estimated_revenue_l365d", lambda x: x.quantile(0.25)),
        revenue_p75=("estimated_revenue_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_occupancy_revenue


In [0]:

# Correlação simples entre ocupação e receita por cidade
summary_occupancy_revenue_corr = (
    df_occ
    .groupby("city")
    .apply(
        lambda g: g["estimated_occupancy_l365d"].corr(g["estimated_revenue_l365d"])
    )
    .reset_index(name="corr_occupancy_revenue")
    .round(3)
)

summary_occupancy_revenue_corr

### Análise - Influência da Ocupação na Receita Anual (apenas anúncios ativos)

Após aplicar o filtro `estimated_occupancy_l365d > 0` e `estimated_revenue_l365d > 0`, a análise passa a refletir exclusivamente anúncios com **demanda e geração de receita efetivas**.  
Com isso, avaliamos de forma mais precisa como a ocupação influencia a produtividade anual.

---

## Resultados - New York City (valores em USD)

| Faixa de Ocupação | Ocupação Média | Receita Média | Mediana |
|-------------------|----------------|----------------|----------|
| 0 – 90 dias | 57.12 | 24,986 | 7,990 |
| 91 – 180 dias | 143.95 | 24,383 | 18,840 |
| 181 – 270 dias | 249.33 | 62,952 | 40,320 |

### Principais observações - NYC

1. Os anúncios com **até 90 dias de ocupação** já apresentam receitas relevantes, indicando forte impacto do **preço unitário elevado**.
2. A faixa **91–180 dias** mantém receita média semelhante à faixa inferior, sugerindo que o aumento da ocupação não se traduz imediatamente em mais receita.
3. Há um **salto expressivo de receita** apenas na faixa **181–270 dias**, quando a ocupação se torna realmente contínua.
4. A correlação entre ocupação e receita é **muito fraca (0.069)**, indicando que:
   - a receita em NYC é explicada majoritariamente pelo **preço da diária**;
   - a ocupação tem impacto secundário.

**Leitura estrutural de NYC:**  
Mesmo com aumento de ocupação, a receita só cresce de forma significativa nos níveis mais altos de uso. O mercado é dominado por anúncios de alto preço, onde poucos dias adicionais de ocupação não alteram substancialmente a receita anual.

---

## Resultados - Rio de Janeiro (valores em BRL)

| Faixa de Ocupação | Ocupação Média | Receita Média | Mediana |
|-------------------|----------------|----------------|----------|
| 0 – 90 dias | 33.82 | 14,809 | 8,316 |
| 91 – 180 dias | 133.50 | 51,608 | 35,292 |
| 181 – 270 dias | 231.82 | 83,003 | 60,000 |

### Principais observações - Rio

1. A relação entre aumento de ocupação e crescimento de receita é **progressiva e consistente**.
2. Da faixa **0–90** para **91–180 dias**, a receita média **mais que triplica**.
3. Na faixa **181–270 dias**, observa-se novamente um crescimento expressivo.
4. A correlação entre ocupação e receita é **moderada (0.328)**, mostrando que:
   - a demanda tem papel central na geração de receita;
   - o preço, isoladamente, não explica a produtividade.

**Leitura estrutural do Rio:**  
A receita anual no Rio é fortemente **sensível ao nível de ocupação**, refletindo um mercado guiado por demanda turística e sazonal.

---

## Comparação entre NYC e Rio

1. Em NYC, a receita cresce de forma **irregular** conforme a ocupação aumenta.
2. No Rio, a receita cresce de forma **quase proporcional** ao aumento da ocupação.
3. A ocupação tem:
   - **baixo poder explicativo da receita em NYC**;
   - **poder explicativo relevante no Rio**.
4. Isso reforça duas lógicas de mercado distintas:
   - **NYC:** mercado orientado a preço.
   - **Rio:** mercado orientado a demanda.

---

## Conclusão da Análise de Ocupação × Receita

A relação entre ocupação e receita anual confirma a diferença estrutural entre os mercados:

- Em **New York City**, a ocupação exerce influência marginal sobre a receita, que é explicada principalmente pelo **valor da diária**.
- No **Rio de Janeiro**, a ocupação é um dos principais motores da receita anual, refletindo a natureza **turística e sazonal** do mercado.

Esse resultado valida empiricamente que os mesmos indicadores operam com **pesos diferentes** nos dois mercados, o que é fundamental para qualquer estratégia de precificação e investimento baseada em Airbnb.

# Conclusões da Pergunta 2 - Quais fatores influenciam o preço da diária e a receita anual?

A Pergunta 2 investigou como diferentes variáveis explicam a formação de preço e os níveis de receita no Airbnb, separando Rio de Janeiro e New York City devido às diferenças de moeda, demanda e estrutura de mercado.  
As análises incluíram: tipo de imóvel, capacidade, reputação, profissionalização do host e ocupação estimada (considerando apenas anúncios ativos).

A seguir estão as conclusões consolidadas.

---

## 1. Tipo de imóvel (room_type) é um dos fatores mais fortes na determinação do preço

- A hierarquia de preços é consistente nas duas cidades:  
  **Entire home/apt > Private room > Shared room.**
- Entire homes apresentam maior variação (dispersão), associada a diferentes padrões de espaço e localização.
- Private rooms têm comportamento intermediário, com preços mais estáveis e menor dispersão.
- Shared rooms concentram a faixa mais econômica.
- A categoria *Hotel room* em NYC contém outliers severos e não deve ser usada sem limpeza adicional.

**Conclusão:**  
Privacidade e estrutura interna do imóvel são determinantes diretos do preço.

---

## 2. Capacidade (accommodates, bedrooms, beds) influencia preço, mas de forma não linear

- `bedrooms` mostrou ser o indicador mais estável e com relação mais clara com o preço.
- `accommodates` tem bom poder explicativo até 6 hóspedes; acima disso, a dispersão cresce e a presença de imóveis de luxo distorce as médias.
- `beds` apresenta relação mais ruidosa, afetada por layouts e configurações diversas.
- Em ambos os mercados, o preço cresce até níveis intermediários de capacidade e depois passa a variar muito.

**Conclusão:**  
Capacidade aumenta preço até certo ponto; imóveis muito grandes têm comportamento de mercado próprio, dominado por nichos de luxo.

---

## 3. Reputação (reviews) tem efeito moderado e secundário sobre o preço

- Ratings são muito concentrados no topo (viés natural do Airbnb), o que reduz diferenças de preço entre faixas.
- Notas mais altas têm preços medianos um pouco maiores, mas não de forma acentuada.
- Reviews por mês têm comportamento misto:  
  - no Rio, maior engajamento está associado a preços mais acessíveis;  
  - em NYC, o efeito é mais disperso.
- Recência de reviews tem relação mais forte com demanda do que com preço.

**Conclusão:**  
Reputação afeta confiança e probabilidade de reserva, mas não é um determinante principal do preço. Atua como fator de ajuste fino, não como motor de precificação.

---

## 4. Profissionalização do host influencia a distribuição, mas não define preço de forma sistemática

- Ser Superhost não implica preços maiores. Em muitos casos, os Superhosts praticam preços ligeiramente menores, sugerindo estratégia de ocupação mais agressiva.
- Hosts profissionais concentram imóveis de maior receita, mas a maioria opera em faixas de preço semelhantes às dos hospedeiros casuais.
- O segmento profissional apresenta alta dispersão: mistura de imóveis econômicos, intermediários e alguns de luxo extremo.

**Conclusão:**  
Profissionalização contribui para consistência operacional, mas não determina preço de forma direta. Segmentos premium existem, mas não representam o padrão geral.

---

## 5. Ocupação é preditor relevante de receita, com comportamento distinto por cidade (apenas anúncios ativos)

- No **Rio de Janeiro**, a correlação entre ocupação e receita é **moderada (0.328)**:  
  imóveis com maior ocupação tendem a gerar receitas significativamente mais altas.
- Em **NYC**, a correlação é **muito fraca (0.069)**:  
  a receita é explicada predominantemente pelo **preço da diária**, e não pela ocupação.
- No Rio, a receita cresce de forma progressiva conforme a ocupação aumenta.
- Em NYC, a receita só cresce de forma relevante nos níveis mais altos de ocupação (181–270 dias).

**Conclusão:**  
Ocupação é um motor fundamental de receita no Rio, mas exerce papel secundário em NYC, onde o preço unitário tem maior peso.

---

## Conclusão Geral da Pergunta 2

A formação de preço no Airbnb é multifatorial, mas os fatores mais decisivos são estruturais:

1. **Tipo de imóvel**  
2. **Capacidade (especialmente bedrooms)**  
3. **Localização** (analisada em profundidade na Pergunta 3)

Variáveis comportamentais - reputação, engajamento e profissionalização - afetam o preço de forma mais sutil.  
A **receita anual depende tanto do preço quanto da ocupação**, porém com pesos distintos:

- **Rio de Janeiro:** mercado orientado à demanda (ocupação).
- **New York City:** mercado orientado ao preço (valor da diária).

Essa diferença estrutural reforça que estratégias de precificação e investimento devem ser **adaptadas ao perfil específico de cada cidade**.


# Pergunta 3 - Como a localização afeta preço, demanda e competitividade?

### Objetivo da análise

Investigar como a localização geográfica influencia o desempenho dos anúncios no Airbnb, considerando os mercados do Rio de Janeiro e de New York City.

Os objetivos específicos são:

- preço das diárias (`price_numeric`),
- demanda medida pela ocupação estimada (`estimated_occupancy_l365d`),
- receita estimada anual (`estimated_revenue_l365d`),
- padrões locais de competitividade.


A meta é entender o papel da localização como determinante de preço, demanda e retorno, consolidando a dimensão espacial do MVP.

---

### Estratégia da análise

1. Selecionar, para cada cidade, os **10 bairros com maior ocupação média**:
   - utilizar `estimated_occupancy_l365d` para calcular a ocupação média por `neighbourhood_cleansed`;
   - filtrar os Top 10 bairros por cidade com base nessa métrica;
   - garantir que os bairros selecionados tenham volume suficiente de anúncios para análise estável.

2. Para esses Top 10 bairros de cada cidade, comparar:
   - preço médio, mediana e faixas de preço;
   - ocupação média e distribuição de ocupação;
   - receita anual média e mediana;
   - quantidade de anúncios (tamanho da oferta).

3. Avaliar a competitividade local:
   - identificar quais bairros concentram maior receita e maior ocupação;
   - distinguir áreas mais voltadas a tickets altos versus áreas de alta rotatividade;
   - analisar possíveis diferenças estruturais entre Rio e NYC.

4. Integrar os resultados:
   - relacionar localização com preço, demanda e receita;
   - sintetizar como a geografia da cidade molda a competitividade dos anúncios;
   - preparar conclusões que conectem esta pergunta às análises anteriores (tipo de imóvel, capacidade, reputação e profissionalização do host).

---

### Próximo passo

Calcular os Top 10 bairros por ocupação média em cada cidade e utilizá-los como base para as análises de localização da Pergunta 3.


In [0]:
# Cópia da base
df_loc = df_listings.copy()

# Garante apenas anúncios com ocupação válida
df_loc = df_loc[df_loc["estimated_occupancy_l365d"].notna()].copy()

# Agrupa por cidade e bairro
summary_neighbourhood_occupancy = (
    df_loc
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        occupancy_mean=("estimated_occupancy_l365d", "mean"),
        occupancy_median=("estimated_occupancy_l365d", "median"),
        occupancy_p25=("estimated_occupancy_l365d", lambda x: x.quantile(0.25)),
        occupancy_p75=("estimated_occupancy_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)


summary_neighbourhood_occupancy

In [0]:
# Seleciona Top 10 por cidade
top10_neighbourhoods_by_occupancy = (
    summary_neighbourhood_occupancy
    .sort_values(["city", "occupancy_mean"], ascending=[True, False])
    .groupby("city")
    .head(10)
    .reset_index(drop=True)
)

top10_neighbourhoods_by_occupancy

## Ajuste metodológico - Por que alterar o critério do ranking?

Inicialmente foi planejado selecionar os **Top 10 bairros com maior ocupação média** em cada cidade.  
Porém, após uma checagem exploratória preliminar, percebemos que:

- alguns bairros possuem **ocupação média muito alta**,  
- **mas contam com pouquíssimos anúncios** - às vezes menos de 10 unidades.

Isso distorce a análise, pois:

- ocupações extremas com baixa amostra não representam o mercado,
- rankings ficam instáveis (qualquer outlier altera drasticamente o bairro),
- e o objetivo aqui é capturar **padrões robustos e representativos**.

---

## Nova estratégia adotada

Para garantir consistência estatística e conclusões confiáveis, será adotado:

### **Top 10 bairros por volume de anúncios (maior número de listings)**

Motivos:

- concentrações maiores permitem estimativas mais sólidas de preço, ocupação e receita;
- bairros mais relevantes em termos de oferta são os que moldam o mercado e a competitividade real;
- facilita a comparação entre Rio e NYC, evitando ruídos de pouca amostra;
- as análises de preço, demanda e receita tornam-se mais representativas.

---

## Estratégia final da análise

1. Selecionar os **10 bairros com maior número de anúncios** em cada cidade.
2. Para esses bairros, comparar:
   - preço médio, mediana e faixas de preço;
   - ocupação média e distribuição de ocupação;
   - receita anual média e mediana.
3. Avaliar padrões de competitividade:
   - áreas de maior procura,
   - áreas com tíquetes mais altos,
   - regiões de maior produtividade anual.
4. Integrar os resultados entre Rio e NYC,
   conectando localização com preço, demanda e retorno.


In [0]:
# Cópia da base
df_loc = df_listings.copy()

# Garante apenas anúncios com bairro definido
df_loc = df_loc[df_loc["neighbourhood_cleansed"].notna()].copy()

# Agrega por cidade e bairro
summary_neighbourhood_count = (
    df_loc
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count")
    )
    .reset_index()
)

# Ordena por cidade e número de anúncios (decrescente)
summary_neighbourhood_count = summary_neighbourhood_count.sort_values(
    ["city", "count_listings"],
    ascending=[True, False]
)

summary_neighbourhood_count

In [0]:
# Seleciona Top 10 bairros por número de anúncios em cada cidade
top10_neighbourhoods_by_count = (
    summary_neighbourhood_count
    .groupby("city")
    .head(10)
    .reset_index(drop=True)
)

top10_neighbourhoods_by_count

### Análise - Preço por Bairro (Top 10 bairros por volume de anúncios)

Agora que selecionamos os 10 bairros mais relevantes de cada cidade - aqueles que concentram a maior parte da oferta no Airbnb -, avaliamos como a localização influencia o preço dos anúncios.

Nesta análise, investigamos:

- preço médio e mediano por bairro;
- dispersão de preço (P25–P75);
- comparação entre Rio de Janeiro e NYC;
- como a geografia do mercado molda diferentes faixas de preço.

A análise considera exclusivamente os bairros pertencentes ao Top 10 de cada cidade, garantindo representatividade estatística.


In [0]:
# Filtra apenas os bairros do Top 10
top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]]

df_geo = (
    df_listings
    .merge(top_bairros, on=["city", "neighbourhood_cleansed"], how="inner")
    .copy()
)

# Cálculo de preço por bairro 
summary_price_neighbourhood = (
    df_geo
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        price_mean=("price_numeric", "mean"),
        price_median=("price_numeric", "median"),
        price_p25=("price_numeric", lambda x: x.quantile(0.25)),
        price_p75=("price_numeric", lambda x: x.quantile(0.75))
    )
    .reset_index()
)

summary_price_neighbourhood.sort_values(["city", "price_median"], ascending=[True, False])


### Uso de Dados Geoespaciais para Mapas Interativos

Para enriquecer a análise de localização e tornar os resultados mais intuitivos, incorporamos visualizações geográficas utilizando a tabela **`dim_neighbourhoods`**, presente na camada Gold. Esta tabela contém a coluna **`geometry_json`**, que armazena a geometria dos bairros em formato GeoJSON, possibilitando reconstruir polígonos reais de cada região.

Com esses polígonos, conseguimos gerar **mapas coropléticos interativos**, onde características como preço médio, ocupação e receita anual podem ser visualizadas espacialmente, revelando padrões que não aparecem claramente apenas em tabelas.

#### Bibliotecas utilizadas

Para manipular e renderizar os dados geoespaciais, utilizamos:

- **GeoPandas**  
  Para converter o `geometry_json` em objetos geométricos e formar um `GeoDataFrame`.

- **Shapely**  
  Para interpretar as geometrias de cada bairro.

- **Plotly Express**  
  Para criar mapas interativos em Mapbox com excelente qualidade visual.

#### Instalação necessária

Como o ambiente padrão do Databricks não inclui o GeoPandas, é necessário instalar:


`pip install geopandas`


In [0]:
pip install geopandas

In [0]:
import geopandas as gpd
import pandas as pd
import json
import plotly.express as px
from shapely.geometry import shape

# Puxar a tabela dim_neighbourhoods
query_listings_perf = """
SELECT *
FROM mvp_airbnb_catalog.default.dim_neighbourhoods
"""

df_neighbourhoods_spark = spark.sql(query_listings_perf)
df_neigh = df_neighbourhoods_spark.toPandas()


# Converte geometry_json -> shapely geometry
df_neigh = df_neigh.copy()
df_neigh["geometry"] = df_neigh["geometry_json"].apply(
    lambda x: shape(json.loads(x)) if pd.notna(x) else None
)

gdf_neigh = gpd.GeoDataFrame(df_neigh, geometry="geometry")

# Prepara a lista de bairros do Top 10 (rename para bater com dim_neighbourhoods)
top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]].copy()
top_bairros = top_bairros.rename(columns={"neighbourhood_cleansed": "neighbourhood"})

# Junta dim_neighbourhoods com Top 10 bairros
gdf_top = gdf_neigh.merge(
    top_bairros,
    on=["city", "neighbourhood"],
    how="inner"
)

# Prepara o summary de preço por bairro para juntar (rename para bater com dim_neighbourhoods)
summary_price_neigh = summary_price_neighbourhood.rename(
    columns={"neighbourhood_cleansed": "neighbourhood"}
).copy()

gdf_map = gdf_top.merge(
    summary_price_neigh,
    on=["city", "neighbourhood"],
    how="inner"
)

# Converte geometry (shapely) em geojson simples para o Plotly
def geom_to_geojson(geom):
    if geom is None:
        return None
    return json.loads(gpd.GeoSeries([geom]).to_json())["features"][0]["geometry"]

gdf_map["geometry_json_export"] = gdf_map["geometry"].apply(geom_to_geojson)

# Função para gerar mapa por cidade e métrica (ex: price_median)
def plot_map(city, metric, title):
    df_city = gdf_map[gdf_map["city"] == city].copy()

    # Cria um FeatureCollection para o Plotly
    features = []
    for idx, row in df_city.iterrows():
        features.append({
            "type": "Feature",
            "geometry": row["geometry_json_export"],
            "properties": {
                "id": int(idx),
                "neighbourhood": row["neighbourhood"]
            }
        })

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    df_city["feature_id"] = df_city.index.astype(int)

    fig = px.choropleth_mapbox(
        df_city,
        geojson=geojson,
        locations="feature_id",
        featureidkey="properties.id",
        color=metric,
        hover_name="neighbourhood",
        hover_data={
            metric: True,
            "count_listings": True
        },
        mapbox_style="carto-positron",
        zoom=10,
        center={
            "lat": df_city.geometry.centroid.y.mean(),
            "lon": df_city.geometry.centroid.x.mean()
        },
        opacity=0.6
    )

    fig.update_layout(
        title=title,
        margin={"r": 0, "t": 40, "l": 0, "b": 0}
    )

    fig.show()

# Representação visual do ranking do RJ
plot_map(
    city="Rio de Janeiro",
    metric="price_mean",
    title="Preço médio por bairro (Top 10) - Rio de Janeiro"
)


In [0]:
# Representacão visual do ranking de NYC
plot_map(
    city="New York City",
    metric="price_mean",
    title="Preço médio por bairro (Top 10) - NYC"
)


### Análise - Preço Médio por Bairro (Top 10 bairros por volume de anúncios)

Nesta primeira etapa da Pergunta 3, investigamos como a localização impacta o preço médio dos anúncios nos bairros mais relevantes de cada cidade - aqueles com maior número de listings, garantindo representatividade estatística.

Os resultados mostram padrões muito distintos entre Rio de Janeiro e New York City, revelando a força da geografia na precificação.

---

## Resultados - New York City

| Bairro | Preço médio (USD) |
|-------|--------------------|
| Midtown | 2360.96 |
| Upper West Side | 1886.68 |
| Hell’s Kitchen | 1567.96 |
| Upper East Side | 1141.95 |
| Williamsburg | 791.91 |
| East Village | 276.60 |
| Crown Heights | 203.50 |
| Harlem | 178.48 |
| Bedford-Stuyvesant | 145.65 |
| Bushwick | 121.20 |

### Principais pontos

- **Midtown**, **Upper West Side** e **Hell’s Kitchen** formam o núcleo premium de NYC, com preços médios muito acima da média geral da cidade.
- Mesmo bairros tradicionalmente residenciais e valorizados, como **Upper East Side**, apresentam valores elevados.
- Bairros como **Williamsburg** também figuram entre os mais caros, consolidando-se como polos culturais valorizados.
- A queda é acentuada em bairros periféricos do Brooklyn, como **Bedford-Stuyvesant**, **Bushwick** e **Crown Heights**, onde os preços médios ficam entre USD 120 e 210.
- A amplitude entre top 10 é enorme - Midtown custa **20× mais** que Bushwick - reforçando como NYC é marcada por **forte desigualdade espacial de preços**.

---

## Resultados - Rio de Janeiro

| Bairro | Preço médio (BRL) |
|-------|---------------------|
| Leblon | 1011.20 |
| Barra da Tijuca | 1009.37 |
| Ipanema | 888.25 |
| Jacarepaguá | 752.90 |
| Recreio dos Bandeirantes | 619.17 |
| Copacabana | 646.78 |
| Flamengo | 550.76 |
| Santa Teresa | 442.61 |
| Botafogo | 427.13 |
| Centro | 254.83 |

### Principais pontos

- Assim como em NYC, a zona mais valorizada é clara: **Leblon** e **Ipanema**, seguidos pela **Barra da Tijuca**, formam o eixo premium da cidade.
- O Rio tem preços médios bem mais concentrados do que NYC: mesmo bairros com grande oferta, como **Botafogo** ou **Flamengo**, mantêm valores medianos próximos.
- **Centro** é o bairro mais barato entre o Top 10, com valores significativamente mais baixos - padrão esperado por conta do perfil comercial e menor atratividade turística.
- A distribuição de preços é mais equilibrada que em NYC: a diferença entre o bairro mais caro (Leblon) e o mais barato (Centro) é de aproximadamente **4×**, contra **20×** em NYC.

---

## Comparação entre NYC e Rio

1. **NYC apresenta disparidades geográficas muito maiores**, com polos extremamente caros (Midtown) e regiões muito mais acessíveis (Bushwick).
2. **O Rio é mais homogêneo**, embora tenha um eixo de luxo bem definido (Leblon–Ipanema–Barra).
3. Em ambas as cidades, o preço acompanha a:
   - proximidade de áreas turísticas,
   - densidade de comércio/serviços,
   - infraestrutura urbana,
   - prestígio histórico do bairro.
4. **NYC tem preços consistentemente mais altos**, mas, além disso, apresenta maior influência da geografia na precificação.

---

## Conclusão da análise

A localização exerce um papel central na precificação dos anúncios.  
Nos dois mercados:

- bairros mais valorizados concentram preços mais altos,
- áreas periféricas ou residenciais apresentam valores menores,
- regiões com maior fluxo turístico são consistentemente mais caras.

No entanto, NYC exibe **variação espacial muito mais acentuada**, indicando um mercado mais segmentado e heterogêneo.  
O Rio de Janeiro, embora também apresente um eixo premium claro, mantém uma distribuição de preços mais uniforme entre os bairros do Top 10.

Esta análise confirma que **a geografia é um dos fatores mais determinantes para a precificação** - e um dos mais diferentes entre os dois mercados.


### Análise - Ocupação Média por Bairro (Top 10 bairros por volume de anúncios)

Na próxima etapa da Pergunta 3, analisaremos como a ocupação estimada varia entre os 10 bairros mais representativos de cada cidade.

O objetivo é identificar:

- quais bairros apresentam maior demanda ao longo do ano;
- diferenças relevantes entre áreas turísticas, residenciais e centrais;
- padrões geográficos de procura no Rio de Janeiro e em New York City;
- a consistência ou variabilidade da ocupação dentro de cada bairro (P25–P75).

Usaremos tanto tabelas estatísticas quanto mapas interativos baseados na `dim_neighbourhoods` para visualizar esses padrões.


In [0]:
# Ocupação por bairro - estatísticas descritivas (Top 10 por cidade)

top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]]

df_geo_occ = (
    df_listings
    .merge(top_bairros, on=["city", "neighbourhood_cleansed"], how="inner")
    .copy()
)

df_geo_occ = df_geo_occ[df_geo_occ["estimated_occupancy_l365d"].notna()].copy()

summary_occupancy_neighbourhood = (
    df_geo_occ
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        occupancy_mean=("estimated_occupancy_l365d", "mean"),
        occupancy_median=("estimated_occupancy_l365d", "median"),
        occupancy_p25=("estimated_occupancy_l365d", lambda x: x.quantile(0.25)),
        occupancy_p75=("estimated_occupancy_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_occupancy_neighbourhood.sort_values(
    ["city", "occupancy_mean"],
    ascending=[True, False]
)



**Ajuste metodológico na métrica de ocupação**

Durante a execução inicial da análise, foi identificado um volume significativo de anúncios com `estimated_occupancy_l365d = 0`, o que distorcia as estatísticas de ocupação média por bairro, especialmente em New York City.

Para tornar a análise mais fiel à **demanda efetiva**, foi aplicado um filtro considerando apenas anúncios com:

`estimated_occupancy_l365d > 0`

Com isso:
- a amostra foi reduzida para anúncios realmente ativos;
- os valores de média e mediana passaram a refletir a ocupação real;
- os padrões espaciais tornaram-se estatisticamente mais coerentes e interpretáveis.

Apesar da redução da amostra, este critério garante maior robustez analítica nos resultados apresentados a seguir.


In [0]:
# Ocupação por bairro - estatísticas descritivas (Top 10 por cidade)

top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]]

df_geo_occ = (
    df_listings
    .merge(top_bairros, on=["city", "neighbourhood_cleansed"], how="inner")
    .copy()
)

df_geo_occ = df_geo_occ[df_geo_occ["estimated_occupancy_l365d"] > 0].copy()

summary_occupancy_neighbourhood = (
    df_geo_occ
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        occupancy_mean=("estimated_occupancy_l365d", "mean"),
        occupancy_median=("estimated_occupancy_l365d", "median"),
        occupancy_p25=("estimated_occupancy_l365d", lambda x: x.quantile(0.25)),
        occupancy_p75=("estimated_occupancy_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_occupancy_neighbourhood.sort_values(
    ["city", "occupancy_mean"],
    ascending=[True, False]
)



In [0]:
# Prepara dim_neighbourhoods com geometria
df_neigh_geo = df_neigh.copy()

df_neigh_geo["geometry"] = df_neigh_geo["geometry_json"].apply(
    lambda x: shape(json.loads(x)) if pd.notna(x) else None
)

gdf_neigh = gpd.GeoDataFrame(df_neigh_geo, geometry="geometry")

# Prepara lista de Top 10 bairros (rename para bater com dim_neighbourhoods)
top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]].copy()
top_bairros = top_bairros.rename(columns={"neighbourhood_cleansed": "neighbourhood"})

# Junta dim_neighbourhoods com top 10
gdf_top = gdf_neigh.merge(
    top_bairros,
    on=["city", "neighbourhood"],
    how="inner"
)

# Junta as estatísticas de ocupação (rename para bater com dim_neighbourhoods)
occ_neigh_renamed = summary_occupancy_neighbourhood.rename(
    columns={"neighbourhood_cleansed": "neighbourhood"}
).copy()

gdf_map = gdf_top.merge(
    occ_neigh_renamed,
    on=["city", "neighbourhood"],
    how="inner"
)

# Converte geometria shapely em geojson simples por linha
def geom_to_geojson(geom):
    if geom is None:
        return None
    return json.loads(gpd.GeoSeries([geom]).to_json())["features"][0]["geometry"]

gdf_map["geometry_json_export"] = gdf_map["geometry"].apply(geom_to_geojson)

# Função genérica para plotar mapa por cidade e métrica
def plot_map(city, metric, title):
    df_city = gdf_map[gdf_map["city"] == city].copy()

    # Cria FeatureCollection
    features = []
    for idx, row in df_city.iterrows():
        if row["geometry_json_export"] is None:
            continue
        features.append({
            "type": "Feature",
            "geometry": row["geometry_json_export"],
            "properties": {
                "id": int(idx),
                "neighbourhood": row["neighbourhood"]
            }
        })

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    df_city["feature_id"] = df_city.index.astype(int)

    fig = px.choropleth_mapbox(
        df_city,
        geojson=geojson,
        locations="feature_id",
        featureidkey="properties.id",
        color=metric,
        hover_name="neighbourhood",
        hover_data={
            metric: True,
            "count_listings": True
        },
        mapbox_style="carto-positron",
        zoom=10,
        center={
            "lat": df_city.geometry.centroid.y.mean(),
            "lon": df_city.geometry.centroid.x.mean()
        },
        opacity=0.6
    )

    fig.update_layout(
        title=title,
        margin={"r": 0, "t": 40, "l": 0, "b": 0}
    )

    fig.show()

# Mapa de ocupação média - Rio de Janeiro
plot_map(
    city="Rio de Janeiro",
    metric="occupancy_mean",
    title="Ocupação média estimada (dias/ano) por bairro - Top 10 - Rio de Janeiro"
)


In [0]:
# Mapa de ocupação média - New York City
plot_map(
    city="New York City",
    metric="occupancy_mean",
    title="Estimated average occupancy (days/year) by neighbourhood - Top 10 - New York City"
)


### Análise - Ocupação Média por Bairro (Top 10 bairros por volume de anúncios | Apenas anúncios com ocupação > 0)

Para evitar distorções causadas por anúncios inativos (com `estimated_occupancy_l365d = 0`), a análise foi recalculada considerando **apenas anúncios com ocupação estritamente maior que zero**.  
Isso permite avaliar a **demanda real efetiva** dos bairros, excluindo anúncios parados ou não utilizados no período.

---

## Resultados - New York City (apenas anúncios ativos)

| Bairro | Ocupação média (dias/ano) | Mediana |
|--------|----------------------------|----------|
| Crown Heights | 174.10 | 180 |
| Bedford-Stuyvesant | 168.59 | 180 |
| Bushwick | 157.74 | 128 |
| Harlem | 156.57 | 166 |
| Williamsburg | 154.56 | 166 |
| East Village | 152.49 | 120 |
| Upper West Side | 150.25 | 120 |
| Upper East Side | 142.39 | 120 |
| Midtown | 139.43 | 120 |
| Hell’s Kitchen | 139.06 | 120 |

### Principais observações - NYC

1. **A ocupação média efetiva é alta quando o anúncio está ativo**, variando entre ~139 e ~174 dias por ano.
2. A **mediana elevada (120–180 dias)** indica uso contínuo ao longo do ano para grande parte dos anúncios ativos.
3. Bairros residenciais do Brooklyn (Crown Heights, Bed-Stuy, Bushwick) lideram a demanda.
4. Bairros turísticos centrais (Midtown, Hell’s Kitchen, Upper East/West Side) apresentam **ocupação menor**, apesar de preços mais altos.
5. Isso revela um padrão de mercado claro:
   - regiões centrais → tíquetes mais altos,
   - regiões residenciais → maior rotatividade e uso efetivo.

**Conclusão para NYC (demanda real):**  
Quando desconsideramos anúncios inativos, a demanda é **forte, estável e bem distribuída**, especialmente em bairros residenciais. A ocupação não acompanha diretamente os bairros de maior preço.

---

## Resultados - Rio de Janeiro (apenas anúncios ativos)

| Bairro | Ocupação média (dias/ano) | Mediana |
|--------|----------------------------|----------|
| Ipanema | 95.19 | 72 |
| Leblon | 90.81 | 66 |
| Copacabana | 88.98 | 60 |
| Centro | 88.06 | 60 |
| Botafogo | 78.98 | 48 |
| Barra da Tijuca | 75.62 | 48 |
| Flamengo | 74.24 | 48 |
| Recreio dos Bandeirantes | 64.59 | 36 |
| Jacarepaguá | 63.11 | 36 |
| Santa Teresa | 58.33 | 30 |

### Principais observações - Rio

1. Ipanema, Leblon, Copacabana e Centro concentram a **maior demanda efetiva**.
2. A ocupação média varia entre ~58 e ~95 dias, inferior à NYC em termos absolutos, mas:
3. A ocupação no Rio está **mais diretamente conectada ao turismo** do que à residência.
4. A Zona Sul e o Centro formam o eixo principal de demanda.
5. A Zona Oeste apresenta ocupação menor e mais sazonal.

---

## Comparação entre NYC e Rio (ocupação efetiva)

1. **Anúncios ativos em NYC têm ocupação média maior** que no Rio.
2. No entanto:
   - NYC possui grande volume de anúncios **inativos**,
   - o Rio apresenta **menos anúncios inativos e demanda mais homogênea**.
3. Em NYC:
   - bairros residenciais = maior demanda,
   - bairros turísticos = maior preço, menor ocupação.
4. No Rio:
   - bairros turísticos = maior preço **e** maior demanda.
5. Isso indica dois modelos distintos de mercado:
   - **NYC:** orientado a long stays e hospedagem residencial recorrente.
   - **Rio:** orientado a turismo e estadias mais concentradas.

---

## Conclusão da análise

Ao filtrar apenas anúncios com ocupação positiva, a análise revela com clareza que:

- A **localização influencia fortemente a demanda**, mas de formas diferentes em cada cidade.
- Em NYC, a ocupação é liderada por bairros residenciais de aluguel recorrente.
- No Rio, a ocupação é dominada por bairros turísticos da Zona Sul e pelo Centro.
- O comportamento de demanda é estruturalmente diferente nos dois mercados, reforçando:
  - a maturidade do mercado residencial em NYC,
  - o perfil turístico concentrado do Rio de Janeiro.

Esta etapa consolida a dimensão espacial da **demanda real** no MVP.


### Análise - Receita Anual Estimada por Bairro (Top 10 bairros por volume de anúncios)

Nesta etapa da Pergunta 3, investigaremos como a **receita anual estimada** se distribui entre os 10 bairros mais relevantes de cada cidade.

O objetivo é entender:

- quais bairros geram maior receita média e mediana ao longo do ano;
- como a produtividade anual varia entre regiões turísticas, residenciais e centrais;
- se existe concentração de receita em poucos bairros ou se há distribuição mais homogênea;
- diferenças estruturais entre Rio de Janeiro e New York City na geração de receita por localização.

Assim como nas análises anteriores, utilizaremos tabelas estatísticas e mapas geoespaciais construídos a partir da `dim_neighbourhoods` para facilitar a comparação visual entre os bairros.


In [0]:
# Receita anual por bairro - estatísticas descritivas (Top 10 por cidade)

top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]]

df_geo_rev = (
    df_listings
    .merge(top_bairros, on=["city", "neighbourhood_cleansed"], how="inner")
    .copy()
)

# Mantém apenas anúncios com receita estimada válida
df_geo_rev = df_geo_rev[df_geo_rev["estimated_revenue_l365d"].notna()].copy()

summary_revenue_neighbourhood = (
    df_geo_rev
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        revenue_mean=("estimated_revenue_l365d", "mean"),
        revenue_median=("estimated_revenue_l365d", "median"),
        revenue_p25=("estimated_revenue_l365d", lambda x: x.quantile(0.25)),
        revenue_p75=("estimated_revenue_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_revenue_neighbourhood.sort_values(
    ["city", "revenue_mean"],
    ascending=[True, False]
)


**Ajuste metodológico na métrica de receita anual**

De forma análoga ao que foi observado na análise de ocupação, identificou-se também a presença de diversos anúncios com `estimated_revenue_l365d = 0`, o que distorcia as estatísticas de receita por bairro e inviabilizava uma interpretação fiel da produtividade real das regiões.

Para garantir consistência metodológica e maior robustez analítica, foi aplicado o mesmo critério de filtragem utilizado anteriormente, considerando apenas anúncios com:

`estimated_revenue_l365d > 0`

Com esse ajuste:
- a análise passa a refletir somente anúncios efetivamente ativos em geração de receita;
- as métricas de média, mediana e dispersão tornam-se representativas da produtividade real;
- mantém-se a comparabilidade com a análise de ocupação.

Os resultados apresentados a seguir consideram exclusivamente essa base filtrada.


In [0]:
# Receita anual por bairro - estatísticas descritivas (Top 10 por cidade)
# Considerando apenas anúncios com receita estimada > 0

top_bairros = top10_neighbourhoods_by_count[["city", "neighbourhood_cleansed"]]

df_geo_rev = (
    df_listings
    .merge(top_bairros, on=["city", "neighbourhood_cleansed"], how="inner")
    .copy()
)

# Mantém apenas anúncios com receita estimada positiva
df_geo_rev = df_geo_rev[df_geo_rev["estimated_revenue_l365d"] > 0].copy()

summary_revenue_neighbourhood = (
    df_geo_rev
    .groupby(["city", "neighbourhood_cleansed"])
    .agg(
        count_listings=("listing_id", "count"),
        revenue_mean=("estimated_revenue_l365d", "mean"),
        revenue_median=("estimated_revenue_l365d", "median"),
        revenue_p25=("estimated_revenue_l365d", lambda x: x.quantile(0.25)),
        revenue_p75=("estimated_revenue_l365d", lambda x: x.quantile(0.75))
    )
    .round(2)
    .reset_index()
)

summary_revenue_neighbourhood.sort_values(
    ["city", "revenue_mean"],
    ascending=[True, False]
)


In [0]:
# Junta estatísticas de receita com a base geoespacial

rev_neigh_renamed = summary_revenue_neighbourhood.rename(
    columns={"neighbourhood_cleansed": "neighbourhood"}
).copy()

gdf_map = gdf_map.drop(
    columns=["revenue_mean", "revenue_median", "revenue_p25", "revenue_p75"],
    errors="ignore"
)

gdf_map = gdf_map.merge(
    rev_neigh_renamed[
        ["city", "neighbourhood", "revenue_mean", "revenue_median", "revenue_p25", "revenue_p75"]
    ],
    on=["city", "neighbourhood"],
    how="left"
)

# Mapa de receita anual média - Rio de Janeiro
plot_map(
    city="Rio de Janeiro",
    metric="revenue_mean",
    title="Receita anual média estimada por bairro - Top 10 - Rio de Janeiro"
)


In [0]:
# Mapa de receita anual média - New York City
plot_map(
    city="New York City",
    metric="revenue_mean",
    title="Receita anual média estimada por bairro (USD) - Top 10 - New York City"
)


### Análise - Receita Anual Estimada por Bairro (Top 10 | apenas anúncios ativos)

Após o ajuste metodológico para excluir anúncios com `estimated_revenue_l365d = 0`, analisamos a **produtividade anual efetiva** dos 10 bairros mais representativos de cada cidade.  
Essa métrica integra **preço e ocupação**, refletindo diretamente o potencial financeiro real de cada região.

---

## Resultados - New York City (valores em USD)

| Bairro | Receita média | Mediana |
|--------|----------------|----------|
| Midtown | 89,910 | 34,305 |
| Hell’s Kitchen | 54,148 | 20,400 |
| Upper West Side | 51,780 | 28,800 |
| East Village | 37,459 | 28,500 |
| Crown Heights | 36,946 | 23,575 |
| Upper East Side | 36,793 | 23,400 |
| Williamsburg | 32,466 | 26,640 |
| Bedford-Stuyvesant | 30,341 | 22,688 |
| Harlem | 29,260 | 18,198 |
| Bushwick | 24,597 | 18,720 |

### Principais observações - NYC

1. **Midtown é um outlier absoluto de produtividade**, com receita média anual muito superior aos demais bairros.
2. Bairros centrais e turísticos (Midtown, Hell’s Kitchen, Upper West Side, East Village) dominam o topo do ranking.
3. Bairros residenciais do Brooklyn (Crown Heights, Williamsburg, Bed-Stuy, Bushwick) apresentam:
   - receitas médias menores,
   - mas medianas relativamente estáveis.
4. Mesmo com alta ocupação efetiva em bairros residenciais, a **receita anual é mais fortemente explicada pelo preço do que pela ocupação**.
5. As receitas medianas em boa parte dos bairros permanecem entre **USD 18k e USD 30k**, mostrando que poucos anúncios concentram grande parte da receita total.

**Leitura estrutural de NYC:**  
A produtividade anual é fortemente **concentrada em regiões premium**, e o preço é o principal motor da receita, mais do que a ocupação.

---

## Resultados - Rio de Janeiro (valores em BRL)

| Bairro | Receita média | Mediana |
|--------|----------------|----------|
| Leblon | 61,956 | 36,660 |
| Ipanema | 59,761 | 35,442 |
| Barra da Tijuca | 36,798 | 22,257 |
| Copacabana | 33,544 | 19,920 |
| Botafogo | 27,140 | 12,840 |
| Flamengo | 26,768 | 13,500 |
| Recreio dos Bandeirantes | 22,057 | 10,656 |
| Jacarepaguá | 21,762 | 9,720 |
| Centro | 16,026 | 10,545 |
| Santa Teresa | 14,960 | 6,936 |

### Principais observações - Rio

1. **Leblon e Ipanema são os bairros mais produtivos do Rio**, em média e mediana.
2. Barra da Tijuca e Copacabana aparecem como polos intermediários, com alta oferta e boa produtividade.
3. Botafogo e Flamengo mantêm receitas estáveis, porém inferiores às áreas mais turísticas.
4. Zona Oeste (Recreio, Jacarepaguá) e Santa Teresa apresentam:
   - menor produtividade,
   - maior dispersão,
   - maior dependência de sazonalidade.
5. O Centro tem produtividade moderada, reforçando seu perfil mais volátil (corporativo, eventos, curta permanência).

**Leitura estrutural do Rio:**  
A receita anual acompanha de forma clara os **bairros turísticos de alto padrão** e também reflete a **ocupação elevada** dessas regiões.

---

## Comparação entre NYC e Rio

1. Em termos absolutos:
   - NYC gera receitas médias muito superiores (valores em USD).
   - O Rio apresenta valores mais baixos em BRL, porém com:
     - maior alinhamento entre demanda e receita.
2. Em NYC:
   - a receita é puxada pelo **preço alto**,
   - a ocupação tem papel secundário.
3. No Rio:
   - a receita resulta da **combinação de preço + alta demanda turística**.
4. A dispersão em NYC é maior:
   - grande distância entre Midtown e os bairros residenciais.
5. No Rio, a diferença entre bairros é menor, criando um mercado mais homogêneo em termos de produtividade regional.

---

## Conclusão da análise de receita

A receita anual por bairro evidencia modelos econômicos distintos:

- **NYC:** mercado altamente financeiro, dependente de preço e localização premium.
- **Rio:** mercado turístico, fortemente sensível à demanda e sazonalidade.

Essa leitura complementa de forma direta as análises de **preço** e **ocupação**, permitindo agora consolidar a **competitividade geográfica** na conclusão geral da Pergunta 3.


## Conclusão Geral - Pergunta 3  
**Como a localização afeta preço, demanda e competitividade?**

A análise geográfica baseada nos **Top 10 bairros por volume de anúncios** permitiu avaliar de forma integrada como **preço, ocupação, receita anual e concentração de oferta** se distribuem espacialmente em Rio de Janeiro e New York City. Os resultados evidenciam **dois modelos de mercado estruturalmente distintos**.

---

### 1. Impacto da localização no **preço**

- Em **NYC**, os preços médios mais elevados concentram-se em bairros centrais e turísticos como:
  - *Midtown, Hell’s Kitchen, Upper West Side e Upper East Side*.
- Em **Rio de Janeiro**, os preços mais altos estão fortemente associados à **Zona Sul**:
  - *Leblon, Ipanema e Barra da Tijuca* lideram os valores médios.
- A localização é, em ambas as cidades, o principal determinante do **nível de preço**, criando zonas claramente “premium”.

---

### 2. Impacto da localização na **demanda (ocupação efetiva)**

- Em **NYC**, após excluir anúncios com ocupação zero:
  - os bairros com **maior ocupação efetiva são residenciais**, como *Crown Heights, Bedford-Stuyvesant, Bushwick e Harlem*;
  - bairros centrais turísticos apresentam **ocupação menor**, apesar de preços mais altos.
- No **Rio de Janeiro**, a ocupação é:
  - mais alta nos **bairros turísticos da Zona Sul e no Centro** (*Ipanema, Leblon, Copacabana, Centro*);
  - significativamente menor na Zona Oeste (*Recreio, Jacarepaguá*).
- Isso indica que:
  - o **Rio tem um padrão de demanda claramente turístico**;
  - NYC tem um padrão mais **residencial e recorrente**.

---

### 3. Impacto da localização na **receita anual**

- Em **NYC**, a receita anual é fortemente concentrada em bairros de alto preço:
  - *Midtown* é um outlier absoluto;
  - bairros premium dominam a produtividade mesmo com menor ocupação.
- No **Rio**, a receita acompanha de forma mais direta:
  - a **demanda turística**;
  - a combinação entre **preço e ocupação** em bairros como *Leblon, Ipanema e Copacabana*.
- Em NYC, a **receita é explicada majoritariamente pelo preço**.
- No Rio, a **receita é explicada pelo binômio preço + ocupação**.

---

### 4. Volume de anúncios e competitividade

- O **volume de anúncios** concentra-se:
  - em bairros residenciais densos em NYC (*Bedford-Stuyvesant, Williamsburg*);
  - em bairros turísticos consolidados no Rio (*Copacabana, Barra, Ipanema*).
- Em NYC, alta oferta não implica necessariamente alta receita nem alta ocupação.
- No Rio, há maior **alinhamento entre oferta, demanda e receita** nos bairros turísticos.
- Isso gera dois tipos claros de competitividade:
  - **NYC:** competição intensa em áreas residenciais com margens unitárias menores;
  - **Rio:** competição concentrada em zonas turísticas com maior retorno por anúncio.

---

### Síntese final da Pergunta 3

A localização exerce um papel central na estrutura econômica dos anúncios, mas de formas distintas em cada cidade:

- **New York City**
  - mercado orientado a preço,
  - alta segmentação espacial,
  - grande diferença entre zonas premium e residenciais,
  - receita pouco sensível à ocupação.

- **Rio de Janeiro**
  - mercado orientado à demanda turística,
  - forte concentração na Zona Sul e no Centro,
  - maior equilíbrio entre preço, ocupação e receita,
  - elevada sensibilidade à sazonalidade.

Em resumo, a análise confirma que a **dimensão geográfica é determinante para a competitividade**, mas sua influência sobre **preço, demanda e produtividade anual varia substancialmente entre os dois mercados**, refletindo diferenças estruturais profundas no perfil de consumo, turismo e uso habitacional do Airbnb em cada cidade.


# Consolidação Geral das Conclusões da Análise de Dados

Esta etapa teve como objetivo responder, de forma estruturada e baseada na camada Gold, às três perguntas de negócio definidas para o MVP:

1. **O que diferencia anúncios de alta performance?**  
2. **Quais fatores influenciam o preço da diária e a receita anual?**  
3. **Como a localização afeta preço, demanda e competitividade?**

As conclusões abaixo integram os principais achados das três perguntas, conectando performance, precificação e geografia para os mercados de **Rio de Janeiro** e **New York City**.

---

## 1. Estrutura geral de performance: poucos anúncios concentram a maior parte da demanda

- A métrica de performance foi definida como **ocupação estimada nos últimos 365 dias** (`estimated_occupancy_l365d`), com **percentis por cidade**.
- Em ambas as cidades, a distribuição é altamente assimétrica:
  - mediana baixa, com muitos anúncios pouco ocupados;
  - um grupo reduzido de anúncios com ocupação muito alta (cauda longa).
- A partir disso, foram definidos três grupos:
  - **Base Performance** → até o p75;
  - **High Performance** → p75–p90;
  - **Top Performers** → acima do p90.

**Síntese:**  
A maior parte dos anúncios opera com ocupação baixa ou moderada; um grupo relativamente pequeno (Top 10–25%) sustenta a maior parte da demanda. É nesse grupo que se concentram os padrões de alta performance analisados nas demais seções.

---

## 2. Perfil dos anúncios de alta performance (Pergunta 1)

### 2.1 Tipo de imóvel

- Em ambas as cidades, anúncios de **Entire home/apt** estão desproporcionalmente presentes entre **High Performance** e **Top Performers**.
- No **Rio de Janeiro**, essa predominância é ainda mais clara: os melhores anúncios são quase sempre imóveis inteiros.
- Em **NYC**, há espaço para **Private rooms** alcançarem alta performance, especialmente quando bem avaliados e bem localizados.

**Conclusão:**  
Oferecer o imóvel inteiro é uma das características mais recorrentes entre anúncios de alta performance, sobretudo em mercados turísticos como o Rio.

---

### 2.2 Capacidade (accommodates, bedrooms, beds)

- **NYC:**  
  - Alta performance concentrada em unidades **compactas**, para 1–2 hóspedes.  
  - Imóveis maiores existem, mas não dominam os top performers.

- **Rio de Janeiro:**  
  - Alta performance concentrada em imóveis de **porte médio** (3–4 hóspedes).  
  - Unidades muito grandes (7+ hóspedes) têm presença menor entre os anúncios de melhor desempenho.

**Conclusão:**  
A alta performance não está associada a imóveis muito grandes. Em geral, os anúncios mais bem-sucedidos são compactos (NYC) ou médios (RJ), ajustados ao perfil de demanda de cada cidade.

---

### 2.3 Preço e faixas de competitividade

- Anúncios de alta performance tendem a se posicionar em **faixas intermediárias de preço**:
  - NYC: principalmente entre **USD 100–300**.
  - RJ: principalmente entre **BRL 100–500**.
- Grupos de baixa performance apresentam maior dispersão:
  - muitos imóveis muito baratos (pouca atratividade ou baixa qualidade);
  - e outros muito caros (nichos de luxo com baixa ocupação).

**Conclusão:**  
Alta performance está ligada a **preços competitivos e bem calibrados**, e não necessariamente aos preços mais altos ou mais baixos do mercado.

---

### 2.4 Reviews: demanda ativa > nota alta

- Ratings (`review_scores_rating`) são elevados em todos os grupos, com diferenças pequenas entre base e top performers.
- Os fatores que mais diferenciam os grupos são:
  - **reviews_per_month** → muito maior para high perf e top performers;
  - **review_recency_days** → muito menor, indicando atividade constante.

**Conclusão:**  
A performance está mais ligada ao **fluxo contínuo de reservas e avaliações recentes** do que à nota média em si. Reviews funcionam como espelho da demanda ativa.

---

### 2.5 Comportamento e perfil dos hosts

- **Superhosts** são desproporcionalmente mais comuns entre os top performers em ambas as cidades.
- Em **NYC**, top performers tendem a vir de hosts com **portfólios médios** (2–10 anúncios), indicando foco e qualidade na gestão.
- No **Rio**, os melhores resultados aparecem entre **semi-profissionais e profissionais** (2+ anúncios), sem concentração nos extremos.

**Conclusão:**  
Experiência e consistência operacional (superhosts, gestão estruturada de portfólio) estão fortemente associados à alta performance. Portfólios gigantes não garantem resultados - o fator decisivo é a qualidade da gestão.

---

## 3. Fatores que influenciam preço e receita (Pergunta 2)

### 3.1 Drivers principais de preço

A formação de preço se mostrou multifatorial, mas com **fatores estruturais predominantes**:

1. **Tipo de imóvel (room_type)**  
   Entire home/apt com preços mais altos; private e shared rooms em faixas mais acessíveis.

2. **Capacidade (especialmente bedrooms)**  
   Capacidade até faixas intermediárias (2–4 hóspedes) aumenta preço de forma mais previsível; imóveis muito grandes entram em nichos específicos.

3. **Localização**  
   Endereços premium (zonas centrais/turísticas) elevam estruturalmente o preço - detalhado na Pergunta 3.

Variáveis comportamentais (reviews, profissionalização) influenciam menos o preço diretamente, atuando mais como ajustes finos.

---

### 3.2 Profissionalização do host e preço

- Superhosts e hosts profissionais concentram muitos dos imóveis de maior receita.
- No entanto, seus preços não são sistematicamente mais altos que os de hosts casuais.
- Muitas vezes, praticam preços ligeiramente menores com o objetivo de:
  - manter alta ocupação,
  - aumentar visibilidade,
  - maximizar receita no longo prazo.

**Conclusão:**  
Profissionalização não é sinônimo de “cobrar mais caro”, e sim de **equilibrar preço, ocupação e reputação** para maximizar resultados.

---

### 3.3 Ocupação × Receita (apenas anúncios ativos)

Após filtrar para `estimated_occupancy_l365d > 0` e `estimated_revenue_l365d > 0`:

- **Rio de Janeiro:**
  - correlação **moderada (≈ 0.33)** entre ocupação e receita;
  - aumento de ocupação se traduz em ganho consistente de receita.
- **NYC:**
  - correlação **fraca (≈ 0.07)**;
  - receita é explicada majoritariamente pelo **preço da diária**, não pela ocupação.

**Conclusão:**  
- No **Rio**, a receita é claramente dependente da **demanda** (ocupação).  
- Em **NYC**, a receita é dominada pelo **nível de preço** - pequenas variações de ocupação não alteram tanto o resultado anual.

---

## 4. Papel da localização na competitividade (Pergunta 3)

A análise da Pergunta 3 foi feita a partir dos **Top 10 bairros por volume de anúncios** em cada cidade, combinando:

- preço médio,
- ocupação média,
- receita anual,
- volume de anúncios,
- e, quando possível, mapas baseados na `dim_neighbourhoods` (geometry_json).

### 4.1 Preço por bairro

- **NYC:**
  - Bairros centrais (*Midtown, Hell’s Kitchen, Upper West Side, Upper East Side*) apresentam os preços mais altos.
  - Bairros residenciais do Brooklyn (*Bedford-Stuyvesant, Bushwick, Crown Heights*) têm preços menores e mais homogêneos.

- **Rio de Janeiro:**
  - Preços mais altos concentrados na **Zona Sul e Barra**:
    - *Leblon, Ipanema, Barra da Tijuca*.
  - Bairros como *Copacabana, Botafogo, Flamengo* formam faixa intermediária.
  - *Centro, Jacarepaguá, Santa Teresa, Recreio* apresentam preços mais baixos em média.

**Conclusão:**  
Localização é um driver claro de preço, com zonas premium bem definidas em ambas as cidades.

---

### 4.2 Ocupação por bairro

- Em **NYC**, após filtrar anúncios com ocupação > 0:
  - bairros residenciais como *Bedford-Stuyvesant, Crown Heights, Bushwick, Harlem* apresentam ocupação média elevada;
  - bairros mais caros e centrais nem sempre são os mais ocupados.

- No **Rio**, a ocupação é mais alta em:
  - bairros turísticos da **Zona Sul** (*Copacabana, Ipanema, Leblon*) e **Centro*;
  - reduzindo-se em Zona Oeste e áreas menos turísticas.

**Conclusão:**  
- Em **NYC**, há um descolamento entre localização premium (preço) e níveis máximos de ocupação.  
- No **Rio**, preço e ocupação se alinham mais fortemente nas zonas turísticas tradicionais.

---

### 4.3 Receita anual por bairro

Considerando apenas anúncios com `estimated_revenue_l365d > 0`:

- **NYC:**
  - *Midtown* é um outlier de produtividade, com receita média muito alta.
  - Bairros premium próximos também se destacam, mas há heterogeneidade.
  - Bairros residenciais têm boa ocupação, porém receita menor em termos absolutos.

- **Rio de Janeiro:**
  - *Leblon* e *Ipanema* lideram em receita média e mediana.
  - *Barra da Tijuca* e *Copacabana* compõem um segundo nível de produtividade.
  - Outros bairros têm receitas menores e maior dependência de sazonalidade.

**Conclusão:**  
- Em **NYC**, a receita anual está fortemente concentrada em bairros de preço alto.  
- No **Rio**, a receita combina **preço + alta demanda turística**, com padrão mais distribuído entre bairros turísticos.

---

## 5. Síntese final da Etapa de Análise de Dados

Integrando as três perguntas, os principais aprendizados são:

1. **Performance (ocupação) é concentrada em uma minoria de anúncios**  
   Esses anúncios compartilham padrões recorrentes de estrutura, preço, reviews e gestão.

2. **Tipo de imóvel e capacidade são drivers estruturais**  
   - Entire home/apt domina a alta performance.  
   - Imóveis compactos (NYC) e de porte médio (RJ) são os mais eficientes.

3. **Preço competitivo em faixas intermediárias é chave para alta performance**  
   Nem os mais baratos, nem os mais caros - mas aqueles equilibrados com a expectativa do mercado-alvo de cada cidade.

4. **Reputação importa mais pela atividade (reviews recentes e frequentes) do que pela nota absoluta**  
   Top performers mantêm fluxo contínuo de reviews, refletindo demanda forte e constante.

5. **Hosts experientes (superhosts, semi-profissionais e profissionais) têm papel central**  
   A profissionalização melhora consistência e probabilidade de alta performance, especialmente quando associada a portfólios bem geridos.

6. **Localização é determinante, mas com lógicas distintas**  
   - NYC: mercado mais financeiro e orientado a preço, com grandes diferenças entre zonas premium e residenciais.  
   - Rio: mercado mais turístico e orientado à demanda, com forte papel da Zona Sul e do Centro na combinação preço + ocupação + receita.

---

## Encerramento da Exploração Analítica

Com esta consolidação:

- A **Pergunta 1** (perfil dos anúncios de alta performance) foi respondida com base em ocupação, tipo de imóvel, capacidade, preço, reviews e comportamento dos hosts.
- A **Pergunta 2** (drivers de preço e receita) foi tratada separando fatores estruturais (tipo, capacidade, localização) e comportamentais (reviews, profissionalização, ocupação).
- A **Pergunta 3** (impacto da localização) foi explorada com foco nos principais bairros de cada cidade, incluindo análises numéricas e mapas geográficos.
